In [1]:
import sys
sys.path.append('../scripts')

import os
from utils import cleanCmpnyNm, getGrpCmpnyDict, convertCmpnyNm, convertCmpnyNm
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from time import sleep
import re

# 2019

In [2]:
year = 2019
dfCmpnySumry = pd.read_excel(f'../data/cmpnySumry/cmpnySumry{year}Kor.xlsx', thousands=',')
dfCmpnySumry.head()

,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,자산총액,부채총액,자본총액,자본금,매출액,당기순이익,구분,법인등록번호,year2,rcmgCode
0,201905,2018,DB,디비생명보험(주),이태운,1989/04/14,1997/01/03,K65,보험 및 연금업,563,...,10887144.0,10296289.0,590855.0,193331.0,2180825.0,24637.0,금융보험업,1101110623606,2019,821420.0
1,201905,2018,DB,디비금융서비스(주),황원기,2014/01/13,2014/03/01,K66,금융 및 보험관련 서비스업,30,...,6057.0,2460.0,3597.0,14001.0,13311.0,-2499.0,금융보험업,1101115314474,2019,23044292.0
2,201905,2018,DB,디비에프아이에스(주),강운식,2014/12/01,2018/06/01,J62,"컴퓨터 프로그래밍, 시스템 통합 및 관리업",217,...,47579.0,5854.0,41725.0,1000.0,47269.0,12063.0,일반회사,1101115576727,2019,24038019.0
3,201905,2018,DB,디비금융투자(주),고원종,1982/12/20,1982/12/20,K66,금융 및 보험관련 서비스업,892,...,5428689.0,4794353.0,634336.0,212232.0,725156.0,46854.0,금융보험업,1101110339956,2019,821640.0
4,201905,2018,DB,디비인베스트(주),최경진,2009/11/16,2010/01/04,M71,전문 서비스업,1,...,14912.0,17943.0,-3031.0,306500.0,933.0,-267.0,일반회사,1101114220359,2019,16003715.0


In [3]:
dfCmpnySumry['매출액Val'] = dfCmpnySumry['매출액']
dfCmpnySumry['매출회사id'] = dfCmpnySumry['rcmgCode']
dfCmpnySumry['매입회사id'] = dfCmpnySumry['rcmgCode']
dfCmpnySumry['매출사공개여부'] = ~dfCmpnySumry['기업공개일'].isna()
dfCmpnySumry['매출사상/비'] = dfCmpnySumry['매출사공개여부'].apply(lambda b: '상장' if b else '비상장')
dfCmpnySumry['매입사상/비'] = dfCmpnySumry['매출사상/비']
dfCmpnySumry['매출사금융여부'] = dfCmpnySumry['업종코드'].isin(['K64', 'K65', 'K66'])
dfCmpnySumry['매출사금융/비금융'] = dfCmpnySumry['매출사금융여부'].apply(lambda b: '금융' if b else '비금융')
dfCmpnySumry['매입사금융/비금융'] = dfCmpnySumry['매출사금융/비금융']

In [4]:
dfCmpnySumry['매출회사'] = cleanCmpnyNm(dfCmpnySumry['소속회사명'])
dfCmpnySumry['매입회사'] = cleanCmpnyNm(dfCmpnySumry['소속회사명'])

In [5]:
dfCmpnySumry[dfCmpnySumry['rcmgCode'].isna()]

,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,매출회사id,매입회사id,매출사공개여부,매출사상/비,매입사상/비,매출사금융여부,매출사금융/비금융,매입사금융/비금융,매출회사,매입회사
1962,201905,2018,네이버,(주)락스퀘어,조성환,2017/04/28,2018/03/01,J58,출판업,30,...,NaN,NaN,False,비상장,비상장,False,비금융,비금융,락스퀘어,락스퀘어


In [6]:
dfCmpnySumry.head()

,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,매출회사id,매입회사id,매출사공개여부,매출사상/비,매입사상/비,매출사금융여부,매출사금융/비금융,매입사금융/비금융,매출회사,매입회사
0,201905,2018,DB,디비생명보험(주),이태운,1989/04/14,1997/01/03,K65,보험 및 연금업,563,...,821420.0,821420.0,False,비상장,비상장,True,금융,금융,디비생명보험,디비생명보험
1,201905,2018,DB,디비금융서비스(주),황원기,2014/01/13,2014/03/01,K66,금융 및 보험관련 서비스업,30,...,23044292.0,23044292.0,False,비상장,비상장,True,금융,금융,디비금융서비스,디비금융서비스
2,201905,2018,DB,디비에프아이에스(주),강운식,2014/12/01,2018/06/01,J62,"컴퓨터 프로그래밍, 시스템 통합 및 관리업",217,...,24038019.0,24038019.0,False,비상장,비상장,False,비금융,비금융,디비에프아이에스,디비에프아이에스
3,201905,2018,DB,디비금융투자(주),고원종,1982/12/20,1982/12/20,K66,금융 및 보험관련 서비스업,892,...,821640.0,821640.0,True,상장,상장,True,금융,금융,디비금융투자,디비금융투자
4,201905,2018,DB,디비인베스트(주),최경진,2009/11/16,2010/01/04,M71,전문 서비스업,1,...,16003715.0,16003715.0,False,비상장,비상장,False,비금융,비금융,디비인베스트,디비인베스트


In [7]:
dfAppnGroupSttus = pd.read_excel(f"../data/grpSumry/appnGroupSttus/appnGroupSttus{year}.xlsx")
dfAppnGroupSttus = dfAppnGroupSttus[dfAppnGroupSttus['smerNm'].str.len() <= 3]
dfAppnGroupSttus = dfAppnGroupSttus.reset_index().drop(columns=['index'])
dfAppnGroupSttus['repreCmpny'] = cleanCmpnyNm(dfAppnGroupSttus['repreCmpny'])
dfAppnGroupSttus

,unityGrupNm,unityGrupCode,smerNm,repreCmpny,sumCmpnyCo,invstmntLmtt,entrprsCl
0,삼성,K1000032,이재용,삼성전자,62,상호출자제한집단,일반
1,현대자동차,K3000027,정몽구,현대자동차,53,상호출자제한집단,일반
2,에스케이,K1000050,최태원,에스케이,111,상호출자제한집단,일반
3,엘지,K1000051,구광모,엘지,75,상호출자제한집단,일반
4,롯데,K1000027,신동빈,롯데지주,95,상호출자제한집단,일반
5,한화,K1000082,김승연,한화,75,상호출자제한집단,일반
6,지에스,K3000076,허창수,지에스,64,상호출자제한집단,일반
7,현대중공업,K3000052,정몽준,한국조선해양,31,상호출자제한집단,일반
8,신세계,K1000042,이명희,신세계,40,상호출자제한집단,일반
9,한진,K1000081,조원태,한진칼,32,상호출자제한집단,일반


In [12]:
lookUpListBuy = []
lookUpListSell = []

for grp in dfAppnGroupSttus['unityGrupNm']:
    print("="*15, grp, "="*15)
    dfTrans = pd.read_excel(f"../data/transactions/{year}/{grp}.xlsx")
    dfTrans['매출액총계'] = dfTrans['국내매출액'] + dfTrans['해외매출액']
    dfTrans['year2'] = year
    dfTrans['year'] = year - 1
    dfTrans['grpname2'] = grp
    
    
    if grp in ['엘에스', '미래에셋', '태영', 'DB', 'KG']:
        dfTrans.loc[:, ['매출회사', '매입회사']] = convertCmpnyNm(dfTrans[['매출회사', '매입회사']], grp)
    
    dfTransId = dfTrans.merge(dfCmpnySumry[['매출회사', '매출회사id', '매출사상/비', '매출사금융/비금융', '매출액Val']], on='매출회사', how='left')
    dfTransId = dfTransId.merge(dfCmpnySumry[['매입회사', '매입회사id', '매입사상/비', '매입사금융/비금융']], on='매입회사', how='left')
    print(list(dfTransId[dfTransId['매출회사id'].isna()]['매출회사'].unique()))
    lookUpListSell.extend(list(dfTransId[dfTransId['매출회사id'].isna()]['매출회사'].unique()))
    print(list(dfTransId[dfTransId['매입회사id'].isna()]['매입회사'].unique()))
    lookUpListBuy.extend(list(dfTransId[dfTransId['매입회사id'].isna()]['매출회사'].unique()))
    print(dfTransId.head())
    
    if not os.path.exists(f'../data/transactions-mrg/{year}'):
        os.makedirs(f'../data/transactions-mrg/{year}')
    
    dfTransId.to_excel(f"../data/transactions-mrg/{year}/{grp}.xlsx", index=False)
    
    print("="*15, grp, "="*15)

=============== 삼성 ===============
[]
['에스프린팅솔루션']
   매출회사      매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액        해외매출액  \
0  삼성전자      삼성물산   78676      2269950  149436938.0  16821275  153560595.0   
1  삼성전자  삼성바이오에피스       1      2269950  149436938.0  16821275  153560595.0   
2  삼성전자   삼성디스플레이  164404      2269950  149436938.0  16821275  153560595.0   
3  삼성전자       세메스       5      2269950  149436938.0  16821275  153560595.0   
4  삼성전자   삼성전자서비스  456109      2269950  149436938.0  16821275  153560595.0   

         매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융       매출액Val  \
0  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
1  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
2  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
3  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
4  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   


=============== 엘지 ===============
=============== 롯데 ===============
[]
['롯데디에프글로벌', '롯데닷컴', '롯데백화점마산', '롯데시네마', '롯데로지스틱스', '롯데아이티테크']
   매출회사      매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액   매출액총계  \
0  롯데지주      호텔롯데  12284       167669          NaN  255644   7296  262940   
1  롯데지주   롯데면세점제주   1140       167669          NaN  255644   7296  262940   
2  롯데지주    부산롯데호텔   1073       167669          NaN  255644   7296  262940   
3  롯데지주      롯데물산    913       167669          NaN  255644   7296  262940   
4  롯데지주  롯데디에프글로벌      1       167669          NaN  255644   7296  262940   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융    매출액Val        매입회사id  \
0   2019  2018       롯데  310204.0     상장       비금융  262940.0  6.300200e+05   
1   2019  2018       롯데  310204.0     상장       비금융  262940.0  1.801708e+09   
2   2019  2018       롯데  310204.0     상장       비금융  262940.0  6.306830e+05   
3   2019  2018       롯데  310204.0     상장       비금융  262940.0  6.208900e+05   
4   2019  2

=============== 신세계 ===============
=============== 한진 ===============
[]
['한국글로발로지스틱시스템', 'HANJININTERMODALAMERICAINC.', 'HANJINGLOBALLOGISTICS(VIETNAM)Ltd.', "HanjinInt'lCorp.", "QINGDAOHANJINLUHAIINT'LLOGISTICSCO.,LTD", 'EURASIALOGISTICSSERVICELLC', 'HanjinGlobalLogistics(HongKong)Ltd.', 'HANJINGLOBALLOGISTICS(SHANGHAI)LIMITED', 'HanjinGlobalLogistics(Dalian)Ltd.', 'HANJINGLOBALLOGISTICS(SHENZHEN)LIMITED', 'HANJINGLOBALLOGISTICSGUANGZHOUCO.,LTD', 'HANJINGLOBALLOGISTICSEUROPEs.r.o.', 'GLSAP', "WACINT'LLOGISTICS(HKLIMITED)", "WACINT'LLOGISTICS(QINGDAOCOMPANYLIMITED)"]
   매출회사  매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액       해외매출액  \
0  대한항공   한진칼   178       199900       1714.0  1183532  11463333.0   
1  대한항공    한진  3347       199900       1714.0  1183532  11463333.0   
2  대한항공  한국공항  1003       199900       1714.0  1183532  11463333.0   
3  대한항공  정석기업     1       199900       1714.0  1183532  11463333.0   
4  대한항공  한진관광   112       199900       1714.0  1183532  11463333.0   

   

    매출회사       매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액  \
0  현대백화점       한무쇼핑  95935.0       137411          NaN  1376064      0   
1  현대백화점   현대백화점면세점    566.0       137411          NaN  1376064      0   
2  현대백화점     현대그린푸드  19039.0       137411          NaN  1376064      0   
3  현대백화점   현대캐터링시스템     36.0       137411          NaN  1376064      0   
4  현대백화점  씨엔에스푸드시스템     12.0       137411          NaN  1376064      0   

     매출액총계  year2  year grpname2   매출회사id 매출사상/비 매출사금융/비금융     매출액Val  \
0  1376064   2019  2018    현대백화점  86512.0     상장       비금융  1376064.0   
1  1376064   2019  2018    현대백화점  86512.0     상장       비금융  1376064.0   
2  1376064   2019  2018    현대백화점  86512.0     상장       비금융  1376064.0   
3  1376064   2019  2018    현대백화점  86512.0     상장       비금융  1376064.0   
4  1376064   2019  2018    현대백화점  86512.0     상장       비금융  1376064.0   

         매입회사id 매입사상/비 매입사금융/비금융  
0  6.123910e+05    비상장       비금융  
1  1.803206e+09    비상장       비금융  
2  6.201140

[]
['거래계열회사수', '피오르드프로세싱코리아']
  매출회사      매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액  해외매출액    매출액총계  \
0  코오롱  코오롱인더스트리  19150        43413        169.0  58004  169.0  58173.0   
1  코오롱   코오롱플라스틱   2213        43413        169.0  58004  169.0  58173.0   
2  코오롱   코오롱머티리얼   2440        43413        169.0  58004  169.0  58173.0   
3  코오롱    코오롱글로텍   3859        43413        169.0  58004  169.0  58173.0   
4  코오롱      스위트밀     82        43413        169.0  58004  169.0  58173.0   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융   매출액Val      매입회사id  \
0   2019  2018      코오롱  320234.0     상장       비금융  58173.0  16003302.0   
1   2019  2018      코오롱  320234.0     상장       비금융  58173.0    825841.0   
2   2019  2018      코오롱  320234.0     상장       비금융  58173.0  13004456.0   
3   2019  2018      코오롱  320234.0     상장       비금융  58173.0    325732.0   
4   2019  2018      코오롱  320234.0     상장       비금융  58173.0    389116.0   

  매입사상/비 매입사금융/비금융  
0     상장       비금융  
1     상장       비금융  

['한국테크놀로지그룹', '한국타이어앤테크놀로지', '한국아트라스비엑스', '한국카앤라이프', '한국네트웍스', '한국엔지니어링웍스', '한국프리시전웍스']
['한국타이어앤테크놀로지', '한국카앤라이프', '한국네트웍스', '한국테크놀로지그룹', '한국아트라스비엑스', '한국엔지니어링웍스', '한국프리시전웍스']
          매출회사         매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  \
0    한국테크놀로지그룹  한국타이어앤테크놀로지  57504        57553      57553.0   74874   
1    한국테크놀로지그룹      한국카앤라이프     35        57553      57553.0   74874   
2    한국테크놀로지그룹       한국네트웍스      8        57553      57553.0   74874   
3    한국테크놀로지그룹       신양월드레저      6        57553      57553.0   74874   
4  한국타이어앤테크놀로지    한국테크놀로지그룹   1658         2844    2154809.0  915864   

       해외매출액      매출액총계  year2  year grpname2  매출회사id 매출사상/비 매출사금융/비금융  \
0        NaN        NaN   2019  2018    한국타이어     NaN    NaN       NaN   
1        NaN        NaN   2019  2018    한국타이어     NaN    NaN       NaN   
2        NaN        NaN   2019  2018    한국타이어     NaN    NaN       NaN   
3        NaN        NaN   2019  2018    한국타이어     NaN    NaN       NaN   
4  2277365.0  3193229

=============== 네이버 ===============
=============== 태영 ===============
[]
['에스비에스에이엔티', '에스비에스internationalInc', '케이씨피', 'PLANKEntertainment']
   매출회사      매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액   해외매출액  \
0  태영건설   태영인더스트리     148       850294          NaN  1991815  4234.0   
1  태영건설      유니시티  476421       850294          NaN  1991815  4234.0   
2  태영건설    에코시티개발  133049       850294          NaN  1991815  4234.0   
3  태영건설    엠시에타개발  207263       850294          NaN  1991815  4234.0   
4  태영건설  포천바이오에너지    1049       850294          NaN  1991815  4234.0   

       매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융     매출액Val  \
0  1996049.0   2019  2018       태영  501662.0     상장       비금융  1996049.0   
1  1996049.0   2019  2018       태영  501662.0     상장       비금융  1996049.0   
2  1996049.0   2019  2018       태영  501662.0     상장       비금융  1996049.0   
3  1996049.0   2019  2018       태영  501662.0     상장       비금융  1996049.0   
4  1996049.0   2019  2018       태영  501662.0

=============== 유진 ===============
=============== 금호석유화학 ===============
[]
['상해금호일려소료유한공사', '광동금호일려고분자재료유한공사', '남경금포금호화공유한공사', '강소금호양농화공유한공사']
     매출회사     매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액      해외매출액  \
0  금호석유화학  금호피앤비화학  161794       199963      22370.0  1336599  2562974.0   
1  금호석유화학  금호미쓰이화학    7583       199963      22370.0  1336599  2562974.0   
2  금호석유화학  금호미쓰이화학    7583       199963      22370.0  1336599  2562974.0   
3  금호석유화학  금호미쓰이화학    7583       199963      22370.0  1336599  2562974.0   
4  금호석유화학  금호미쓰이화학    7583       199963      22370.0  1336599  2562974.0   

       매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융     매출액Val  \
0  3899573.0   2019  2018   금호석유화학  351350.0     상장       비금융  3899573.0   
1  3899573.0   2019  2018   금호석유화학  351350.0     상장       비금융  3899573.0   
2  3899573.0   2019  2018   금호석유화학  351350.0     상장       비금융  3899573.0   
3  3899573.0   2019  2018   금호석유화학  351350.0     상장       비금융  3899573.0   
4  3899573.0   201

In [13]:
len(lookUpListSell)

14

In [15]:
lookUpListSell

['현대에너지솔루션',
 '돈돈팜',
 '미래에셋파트너스제구호사모투자',
 '싱그린FS',
 '농업회사법인팜앤코',
 '한국테크놀로지그룹',
 '한국타이어앤테크놀로지',
 '한국아트라스비엑스',
 '한국카앤라이프',
 '한국네트웍스',
 '한국엔지니어링웍스',
 '한국프리시전웍스',
 '제이비당진페럼빌전문투자형사모투자',
 '유진에버베스트턴어라운드기업재무안정사모투자']

In [14]:
len(lookUpListBuy)

573

In [17]:
# merge cmpnySumry
lCmpnySumry = []
for year in range(2020, 2022):
    df = pd.read_excel(f'../data/cmpnySumry/cmpnySumry{year}Kor.xlsx')
    lCmpnySumry.append(df)
    print(len(list(df.columns)))
dfCmpnySumry = pd.concat(lCmpnySumry, axis=0)
dfCmpnySumry.head()

23
23


,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,자산총액,부채총액,자본총액,자본금,매출액,당기순이익,구분,법인등록번호,year2,rcmgCode
0,202005,2019,넥슨,(유)가승개발,서춘식,2016/01/21,2016/01/21,R91,스포츠 및 오락관련 서비스업,3,...,"15,208","6,064","9,144",100,0,-465,일반회사,1345140003989,2020,1.802408e+09
1,202005,2019,지에스,(유)가승개발,서춘식,2016/01/21,2016/04/01,R91,스포츠 및 오락관련 서비스업,3,...,"15,208","6,064","9,144",100,NaN,-465,일반회사,1345140003989,2020,1.802408e+09
2,202005,2019,카카오,(유)매디슨카운티의다리,이명한,2016/11/16,2020/02/03,R90,"창작, 예술 및 여가관련 서비스업",0,...,11,28,-17,10,3,2,일반회사,1101140192605,2020,1.803409e+09
3,202005,2019,엘에스,(유)빌드윈문경태양광발전,김선철,2018/12/11,2019/02/01,D35,"전기, 가스, 증기 및 공기 조절 공급업",1,...,331,0,331,331,0,0,일반회사,1754140000582,2020,2.102105e+09
4,202005,2019,엘에스,(유)빌드윈인동태양광발전,김선철,2018/12/11,2019/02/01,D35,"전기, 가스, 증기 및 공기 조절 공급업",1,...,"4,883","4,140",743,756,0,-13,일반회사,1760140002014,2020,2.102105e+09


In [18]:
dfCmpnySumry.shape

(4896, 23)

In [19]:
dfCmpnySumry.columns

Index(['공개년월', '회계년', '기업집단명', '소속회사명', '대표자', '설립일', '계열편입일', '업종코드', '영위업종',
       '종업원수', '결산기일', '결산주총일', '기업공개일', '자산총액', '부채총액', '자본총액', '자본금', '매출액',
       '당기순이익', '구분', '법인등록번호', 'year2', 'rcmgCode'],
      dtype='object')

In [20]:
len(dfCmpnySumry.columns)

23

In [22]:
dfCmpnySumry.columns = ['publicYm', 'fisclYr', 'unityGrupNm', 'entrprsNm', 'rprsntvNm', 'fondDe',\
                          'grinil', 'indutyCode', 'indutyNm', 'ordtmEmplyCo', 'stacntDudt', 'beforeBsnsStacntDe',\
                          'entrprsOthbcDe', 'assetsTotamt', 'debtTotamt', 'caplTotamt', 'caplAmount', 'selngAmount', 'thstrmNtpfAmount',\
                          'reconstrctur', 'jurirno', 'year2', 'rcmgCode']

In [23]:
dfCmpnySumry['entrprsNm'] = cleanCmpnyNm(dfCmpnySumry['entrprsNm'])
dfCmpnySumry['entrprsNm'].nunique()

2833

In [8]:
dfCmpnySumry.to_excel(f'../cmpnySumry2001-2021Eng.xlsx')

In [10]:
dfCmpnySumry.columns

Index(['publicYm', 'fisclYr', 'unityGrupNm', 'entrprsNm', 'rprsntvNm',
       'fondDe', 'grinil', 'indutyCode', 'indutyNm', 'ordtmEmplyCo',
       'stacntDudt', 'beforeBsnsStacntDe', 'entrprsOthbcDe', 'assetsTotamt',
       'debtTotamt', 'caplTotamt', 'caplAmount', 'selngAmount',
       'thstrmNtpfAmount', 'reconstrctur', 'jurirno', 'year2', 'rcmgCode'],
      dtype='object')

In [ ]:
dfCmpnySumry.columns = ['publicYm', 'fisclYr', 'unityGrupNm', 'entrprsNm', 'rprsntvNm', 'fondDe',\
                        'grinil', 'indutyCode', 'indutyNm', 'ordtmEmplyCo', 'stacntDudt', 'beforeBsnsStacntDe',\
                        'entrprsOthbcDe', 'assetsTotamt', 'debtTotamt', 'caplTotamt', 'caplAmount', 'selngAmount', 'thstrmNtpfAmount',\
                        'reconstrctur', 'jurirno', 'year2']

# 2019

In [1]:
import sys
sys.path.append('../scripts')

import api

In [2]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
myApi = api.Api(apiKey)
myApi.getTransTableAll(2019)#, headless=True)

===============삼성===============
No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['삼성전자' '삼성물산' '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스' '삼성디스플레이'
 '삼성코닝어드밴스드글라스' '에스유머티리얼스' '스테코' '세메스' '삼성전자서비스' '삼성전자서비스씨에스' '삼성전자판매'
 '삼성전자로지텍' '수원삼성축' '삼성메디슨' '삼성에스디아이' '삼성전기' '삼성중공업' '에스디플렉스' '제일패션리테일'
 '삼성웰스토리' '시큐아이' '멀티캠퍼스' '에스티엠' '삼성에스디에스' '에스코어' '오픈핸즈' '미라콤아이앤씨'
 '삼성엔지니어링' '에스원' '휴먼티에스에스' '에스원씨알엠' '제일기획' '호텔신라' '신라스테이' '에이치디씨신라면세점'
 '에스비티엠' '삼성경제연' '삼성라이온즈' '삼성화재해상보험' '삼성화재애니카손해사정' '삼성화재서비스손해사정'
 '삼성화재금융서비스보험대리점' '삼성생명보험' '삼성선물' '삼성자산운용' '생보부동산신탁' '삼성액티브자산운용'
 '삼성생명서비스손해사정' '삼성에스알에이자산운용' '삼성생명금융서비스보험대리점' '삼성증권' '삼성카드' '삼성카드고객서비스']
['삼성물산' '삼성바이오에피스' '삼성디스플레이' '세메스' '삼성전자서비스' '삼성전자판매' '삼성메디슨' '삼성에스디아이'
 '삼성전기' '삼성웰스토리' '시큐아이' '삼성에스디에스' '삼성엔지니어링' '에스원' '제일기획' '호텔신라' '삼성화재해상보험'
 '삼성생명보험' '삼성자산운용' '삼성증권' '삼성카드' '삼성전자' '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트'
 '삼성바이오로직스' '삼성코닝어드밴스드글라스' '에스유머티리얼스' '스테코' '삼성전자로지텍' '수원삼

['엘지' '엘지화학' '씨텍' '행복누리' '팜한농' '엘지하우시스' '엘지토스템비엠' '그린누리' '엘지생활건강' '코카콜라음료'
 '한국음료' '해태에이치티비' '더페이스샵' '크린소울' '씨앤피코스메틱스' '캐이엔아이' '에프엠지' '엘지파루크' '밝은누리'
 '태극제약' '제이에스제약' '엘지엠엠에이' '엘지전자' '엘지디스플레이' '나눔누리' '엘지이노텍' '이노위드' '하이프라자'
 '하이엠솔루텍' '하이텔레서비스' '에이스냉동공조' '하이엔텍' '엘지히타치워터솔루션' '하누리' '로보스타' '실리콘웍스'
 '엘지유플러스' '씨에스리더' '아인텔레서비스' '미디어로그' '데이콤크로싱' '씨에스원파트너' '위드유' '엘지씨엔에스'
 '비즈테크파트너스' '행복마루' '에스앤아이코퍼레이션' '곤지암예원' '미래엠' '드림누리' '서브원' '엘지경영개발원'
 '엘지스포츠' '지투알' '에이치에스애드' '엘베스트' '엘지상사' '판토스' '판토스부산신항물류센터' '헬리스타항공' '한울타리']
['엘지화학' '엘지하우시스' '엘지토스템비엠' '엘지생활건강' '더페이스샵' '엘지파루크' '엘지엠엠에이' '엘지전자'
 '엘지디스플레이' '엘지이노텍' '하이프라자' '엘지히타치워터솔루션' '엘지유플러스' '엘지씨엔에스' '에스앤아이코퍼레이션'
 '엘지경영개발원' '엘지스포츠' '에이치에스애드' '엘지상사' '판토스' '엘지엔시스' 'LGHoldingsJapanCo.,Ltd'
 '엘지' '씨텍' '행복누리' '팜한농' '코카콜라음료' '하이엔텍' '서브원' '팜화옹' '곤지암예원' '하우시스이엔지'
 '한국음료' '해태에이치티비' '크린소울' '씨앤피코스메틱스' '캐이엔아이' '에프엠지' '밝은누리' '태극제약' '하이엠솔루텍'
 '씨에스리더' '비즈테크파트너스' '미래엠' '제이에스제약' '하이텔레서비스' '에이스냉동공조' '하누리'
 '엘지퓨얼셀시스템즈코리아' '실리콘웍스' '아인텔레서비스' '미디어로그' '씨에스원파트너' '위드유' '지투알' '엘베스트'
 '

WebDriverException: Message: chrome not reachable
  (Session info: chrome=101.0.4951.64)


In [3]:
dfTransIdMrgd19, lookUpListSell19, lookUpListBuy19 = myApi.mergeTransId(2019)

=============== 삼성 ===============
[]
['에스프린팅솔루션']
   매출회사      매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액        해외매출액  \
0  삼성전자      삼성물산   78676      2269950  149436938.0  16821275  153560595.0   
1  삼성전자  삼성바이오에피스       1      2269950  149436938.0  16821275  153560595.0   
2  삼성전자   삼성디스플레이  164404      2269950  149436938.0  16821275  153560595.0   
3  삼성전자       세메스       5      2269950  149436938.0  16821275  153560595.0   
4  삼성전자   삼성전자서비스  456109      2269950  149436938.0  16821275  153560595.0   

         매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융       매출액Val  \
0  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
1  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
2  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
3  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   
4  170381870.0   2019  2018       삼성  380725.0     상장       비금융  170381870.0   


=============== 엘지 ===============
=============== 롯데 ===============
[]
['롯데디에프글로벌', '롯데닷컴', '롯데백화점마산', '롯데시네마', '롯데로지스틱스', '롯데아이티테크']
   매출회사      매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액   매출액총계  \
0  롯데지주      호텔롯데  12284       167669          NaN  255644   7296  262940   
1  롯데지주   롯데면세점제주   1140       167669          NaN  255644   7296  262940   
2  롯데지주    부산롯데호텔   1073       167669          NaN  255644   7296  262940   
3  롯데지주      롯데물산    913       167669          NaN  255644   7296  262940   
4  롯데지주  롯데디에프글로벌      1       167669          NaN  255644   7296  262940   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융    매출액Val        매입회사id  \
0   2019  2018       롯데  310204.0     상장       비금융  262940.0  6.300200e+05   
1   2019  2018       롯데  310204.0     상장       비금융  262940.0  1.801708e+09   
2   2019  2018       롯데  310204.0     상장       비금융  262940.0  6.306830e+05   
3   2019  2018       롯데  310204.0     상장       비금융  262940.0  6.208900e+05   
4   2019  2

=============== 신세계 ===============
=============== 한진 ===============
[]
['한국글로발로지스틱시스템', 'HANJININTERMODALAMERICAINC', 'HANJINGLOBALLOGISTICS(VIETNAM)Ltd', "HanjinInt'lCorp", "QINGDAOHANJINLUHAIINT'LLOGISTICSCO.,LTD", 'EURASIALOGISTICSSERVICELLC', 'HanjinGlobalLogistics(HongKong)Ltd', 'HANJINGLOBALLOGISTICS(SHANGHAI)LIMITED', 'HanjinGlobalLogistics(Dalian)Ltd', 'HANJINGLOBALLOGISTICS(SHENZHEN)LIMITED', 'HANJINGLOBALLOGISTICSGUANGZHOUCO.,LTD', 'HANJINGLOBALLOGISTICSEUROPEs.r.o', 'GLSAP', "WACINT'LLOGISTICS(HKLIMITED)", "WACINT'LLOGISTICS(QINGDAOCOMPANYLIMITED)"]
   매출회사  매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액       해외매출액  \
0  대한항공   한진칼   178       199900       1714.0  1183532  11463333.0   
1  대한항공    한진  3347       199900       1714.0  1183532  11463333.0   
2  대한항공  한국공항  1003       199900       1714.0  1183532  11463333.0   
3  대한항공  정석기업     1       199900       1714.0  1183532  11463333.0   
4  대한항공  한진관광   112       199900       1714.0  1183532  11463333.0   

        매

=============== 미래에셋 ===============
=============== 현대백화점 ===============
[]
['한섬글로벌', '금강에이앤디', '현대엘이디', '이이에치케이이앤에스', '동삼현대한매고분', '한섬파리', 'HYUNDAILIVARTVINA', 'LIVARTCANADA', 'HyundaiL&CCanadaInc', 'HyundaiL&C(Shanghai)Co.,LTD', 'HyundaiL&CUSA,LLC', 'HyundaiL&CEuropeGmbH', 'AmericanSurfacingMaterials,LLC', '한우공정기계(상해)유한공사', 'EVERDIGMHEMTLLC', 'EVERDIGMMONGOLIALLC', 'EVERDIGMAMERICAINC']
    매출회사       매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액  \
0  현대백화점       한무쇼핑  95935.0       137411          NaN  1376064      0   
1  현대백화점   현대백화점면세점    566.0       137411          NaN  1376064      0   
2  현대백화점     현대그린푸드  19039.0       137411          NaN  1376064      0   
3  현대백화점   현대캐터링시스템     36.0       137411          NaN  1376064      0   
4  현대백화점  씨엔에스푸드시스템     12.0       137411          NaN  1376064      0   

     매출액총계  year2  year grpname2   매출회사id 매출사상/비 매출사금융/비금융     매출액Val  \
0  1376064   2019  2018    현대백화점  86512.0     상장       비금융  1376064.0   
1  1376064   201

=============== 교보생명보험 ===============
=============== 금호아시아나 ===============
[]
['WEIHAIPOINTHOTEL&GOLFRESORT.CO.,LTD', '금호연건(중국)유한공사']
   매출회사        매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액    해외매출액  \
0  금호산업      아시아나항공  27699        37125          NaN  1360477  15758.0   
1  금호산업  금호고속관광[경기]     15        37125          NaN  1360477  15758.0   
2  금호산업  금호고속관광[서울]     19        37125          NaN  1360477  15758.0   
3  금호산업        금호고속   5424        37125          NaN  1360477  15758.0   
4  금호산업       금호리조트    359        37125          NaN  1360477  15758.0   

       매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융     매출액Val  \
0  1376235.0   2019  2018   금호아시아나  610364.0     상장       비금융  1376235.0   
1  1376235.0   2019  2018   금호아시아나  610364.0     상장       비금융  1376235.0   
2  1376235.0   2019  2018   금호아시아나  610364.0     상장       비금융  1376235.0   
3  1376235.0   2019  2018   금호아시아나  610364.0     상장       비금융  1376235.0   
4  1376235.0   2019  2018   금호아시아나  61

[]
['에이디엠이십일', '대원건설산업', '성우종합건설', '온양관광호텔', '삼라희망재단', '삼라자원개발']
    매출회사    매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액     해외매출액     매출액총계  \
0  남선알미늄      우방  5864        16306          NaN  245704  102131.0  347835.0   
1  남선알미늄  티케이케미칼   547        16306          NaN  245704  102131.0  347835.0   
2  남선알미늄    경남모직   371        16306          NaN  245704  102131.0  347835.0   
3  남선알미늄  동아건설산업  5193        16306          NaN  245704  102131.0  347835.0   
4  남선알미늄   에스엠상선  3507        16306          NaN  245704  102131.0  347835.0   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융    매출액Val      매입회사id  \
0   2019  2018       SM  380040.0     상장       비금융  347835.0  13071534.0   
1   2019  2018       SM  380040.0     상장       비금융  347835.0  12099515.0   
2   2019  2018       SM  380040.0     상장       비금융  347835.0    768600.0   
3   2019  2018       SM  380040.0     상장       비금융  347835.0    500135.0   
4   2019  2018       SM  380040.0     상장       비금융  347835.0     39911.0  

=============== 셀트리온 ===============
=============== DB ===============
[]
['대우전자', '대우전자서비스', '디비라이텍', '디비MetalUSAINC', '디비MetalEuropeGmbH']
     매출회사      매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  해외매출액   매출액총계  \
0  디비아이엔씨    디비아이엔씨      0        46302            0  122170  84669  206839   
1  디비아이엔씨      디비메탈   6651        46302            0  122170  84669  206839   
2  디비아이엔씨      디비월드    158        46302            0  122170  84669  206839   
3  디비아이엔씨     디비하이텍  10506        46302            0  122170  84669  206839   
4  디비아이엔씨  디비에프아이에스   1138        46302            0  122170  84669  206839   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융    매출액Val      매입회사id  \
0   2019  2018       DB  352098.0     상장       비금융  206839.0    352098.0   
1   2019  2018       DB  352098.0     상장       비금융  206839.0  12097776.0   
2   2019  2018       DB  352098.0     상장       비금융  206839.0    910813.0   
3   2019  2018       DB  352098.0     상장       비금융  206839.0    350354.0   
4  

[]
['한라엔컴', '한라엠티스']
    매출회사    매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액     해외매출액     매출액총계  \
0  한라홀딩스      위코     27       199614     133618.0  517512  159123.0  676635.0   
1  한라홀딩스  제이제이한라     82       199614     133618.0  517512  159123.0  676635.0   
2  한라홀딩스      한라  26421       199614     133618.0  517512  159123.0  676635.0   
3  한라홀딩스  한라세라지오     28       199614     133618.0  517512  159123.0  676635.0   
4  한라홀딩스  한라지엘에스     73       199614     133618.0  517512  159123.0  676635.0   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융    매출액Val      매입회사id  \
0   2019  2018       한라  135082.0     상장       비금융  676635.0    477894.0   
1   2019  2018       한라  135082.0     상장       비금융  676635.0     12026.0   
2   2019  2018       한라  135082.0     상장       비금융  676635.0    501719.0   
3   2019  2018       한라  135082.0     상장       비금융  676635.0    232207.0   
4   2019  2018       한라  135082.0     상장       비금융  676635.0  13007189.0   

  매입사상/비 매입사금융/비금융 grpname  grpcode

  매출회사   매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액     해외매출액      매출액총계  \
0  넷마블   체리벅스   268      28628.0       1716.0  534029  625837.0  1159866.0   
1  넷마블   크레노바     1      28628.0       1716.0  534029  625837.0  1159866.0   
2  넷마블   퍼니파우   252      28628.0       1716.0  534029  625837.0  1159866.0   
3  넷마블    포플랫   201      28628.0       1716.0  534029  625837.0  1159866.0   
4  넷마블  넷마블네오  1128      28628.0       1716.0  534029  625837.0  1159866.0   

   year2  year grpname2      매출회사id 매출사상/비 매출사금융/비금융     매출액Val      매입회사id  \
0   2019  2018      넷마블  18079411.0     상장       비금융  1159866.0  21097298.0   
1   2019  2018      넷마블  18079411.0     상장       비금융  1159866.0         NaN   
2   2019  2018      넷마블  18079411.0     상장       비금융  1159866.0  23058843.0   
3   2019  2018      넷마블  18079411.0     상장       비금융  1159866.0  15095901.0   
4   2019  2018      넷마블  18079411.0     상장       비금융  1159866.0  19063628.0   

  매입사상/비 매입사금융/비금융    grpname  grpcode  
0    비상장       

In [4]:
# verify group matching
dfTransIdMrgd19[dfTransIdMrgd19['grpcode'].isna()]

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출회사id,매출사상/비,매출사금융/비금융,매출액Val,매입회사id,매입사상/비,매입사금융/비금융,grpname,grpcode


In [16]:
dfTransIdMrgd19[dfTransIdMrgd19['매출회사'] == '싱그린FS']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출회사id,매출사상/비,매출사금융/비금융,매출액Val,매입회사id,매입사상/비,매입사금융/비금융,grpname,grpcode
70,싱그린FS,그린바이텍,285.0,3646.0,3227.0,11437.0,8513.0,19950.0,2019,2018,하림,NaN,NaN,NaN,NaN,178743.0,비상장,비금융,harim,24062
71,싱그린FS,산과들에프앤씨,140.0,3646.0,3227.0,11437.0,8513.0,19950.0,2019,2018,하림,NaN,NaN,NaN,NaN,21099446.0,비상장,비금융,harim,24062
72,싱그린FS,선진에프에스,393.0,3646.0,3227.0,11437.0,8513.0,19950.0,2019,2018,하림,NaN,NaN,NaN,NaN,13013067.0,비상장,비금융,harim,24062
73,싱그린FS,선진햄,444.0,3646.0,3227.0,11437.0,8513.0,19950.0,2019,2018,하림,NaN,NaN,NaN,NaN,159100.0,비상장,비금융,harim,24062
74,싱그린FS,올품,87.0,3646.0,3227.0,11437.0,8513.0,19950.0,2019,2018,하림,NaN,NaN,NaN,NaN,179793.0,비상장,비금융,harim,24062
75,싱그린FS,하림,2296.0,3646.0,3227.0,11437.0,8513.0,19950.0,2019,2018,하림,NaN,NaN,NaN,NaN,16091473.0,상장,비금융,harim,24062
76,싱그린FS,하림식품,1.0,3646.0,3227.0,11437.0,8513.0,19950.0,2019,2018,하림,NaN,NaN,NaN,NaN,23007263.0,비상장,비금융,harim,24062
77,싱그린FS,SinggreenVinaCOMPANYLIMITED,3227.0,3646.0,3227.0,11437.0,8513.0,19950.0,2019,2018,하림,NaN,NaN,NaN,NaN,NaN,NaN,NaN,harim,24062


In [10]:
dfLookUp = pd.read_excel("../data/prev/2001_2018소속회사개요(firmid).xlsx")[['firmname', 'firmid']].drop_duplicates()
dfLookUp['firmname'] = cleanCmpnyNm(dfLookUp['firmname'])
dfLookUp

,firmname,firmid
0,가승개발,1.802408e+09
5,농업회사법인그린피그팜스,1.904345e+07
6,대한토건,2.101673e+07
7,부천신세계,1.902609e+09
9,상신기획,1.304847e+07
...,...,...
23164,흥업,6.140920e+05
23171,흥한레미콘,3.943000e+05
23174,희망세움,1.707444e+07
23175,희성전선,3.803420e+05


In [41]:
dfFindSell = dfTransIdMrgd19[dfTransIdMrgd19['매출회사id'].isna()]
len(dfFindSell)

43

In [42]:
dfFindSell = dfFindSell.merge(dfLookUp, left_on='매출회사', right_on='firmname', how='left')
dfFindSell = dfFindSell[['매출회사', 'firmid']].drop_duplicates()
dfFindSell

,매출회사,firmid
0,현대에너지솔루션,NaN
3,돈돈팜,372671.0
5,미래에셋파트너스제구호사모투자,NaN
6,싱그린FS,NaN
14,농업회사법인팜앤코,NaN
15,한국테크놀로지그룹,NaN
19,한국타이어앤테크놀로지,NaN
26,한국아트라스비엑스,NaN
27,한국카앤라이프,NaN
30,한국네트웍스,NaN


In [43]:
dfFindSell.reset_index(inplace=True)
dfFindSell = dfFindSell.drop(columns=['index'])
dfFindSell

,매출회사,firmid
0,현대에너지솔루션,NaN
1,돈돈팜,372671.0
2,미래에셋파트너스제구호사모투자,NaN
3,싱그린FS,NaN
4,농업회사법인팜앤코,NaN
5,한국테크놀로지그룹,NaN
6,한국타이어앤테크놀로지,NaN
7,한국아트라스비엑스,NaN
8,한국카앤라이프,NaN
9,한국네트웍스,NaN


In [44]:
dfFindSell.loc[0, 'firmid'] = 'IL9297'
dfFindSell.loc[2, 'firmid'] = 'HQ9331'
dfFindSell.loc[3, 'firmid'] = 467992
dfFindSell.loc[4, 'firmid'] = 506576
dfFindSell.loc[5, 'firmid'] = 351067
dfFindSell.loc[6, 'firmid'] = 'I66472'
dfFindSell.loc[7, 'firmid'] = 351296
dfFindSell.loc[8, 'firmid'] = 'IN6515'
dfFindSell.loc[9, 'firmid'] = 153529
dfFindSell.loc[10, 'firmid'] = 767662
dfFindSell.loc[11, 'firmid'] = 350842
dfFindSell.loc[12, 'firmid'] = 'HT9817'
dfFindSell.loc[13, 'firmid'] = 'HI4792'
dfFindSell

,매출회사,firmid
0,현대에너지솔루션,IL9297
1,돈돈팜,372671.0
2,미래에셋파트너스제구호사모투자,HQ9331
3,싱그린FS,467992
4,농업회사법인팜앤코,506576
5,한국테크놀로지그룹,351067
6,한국타이어앤테크놀로지,I66472
7,한국아트라스비엑스,351296
8,한국카앤라이프,IN6515
9,한국네트웍스,153529


In [45]:
dfCharToInt = pd.read_excel('../data/utils/char-to-int.xlsx')
dfFindSell['매출회사id'] = dfFindSell['firmid']
for i in range(len(dfCharToInt)):
    dfFindSell['매출회사id'] = dfFindSell['매출회사id'].replace(dfCharToInt.iloc[i, 0], str(dfCharToInt.iloc[i, 1]), regex=True)
dfFindSell

,매출회사,firmid,매출회사id
0,현대에너지솔루션,IL9297,1902309297
1,돈돈팜,372671.0,372671.0
2,미래에셋파트너스제구호사모투자,HQ9331,1802809331
3,싱그린FS,467992,467992
4,농업회사법인팜앤코,506576,506576
5,한국테크놀로지그룹,351067,351067
6,한국타이어앤테크놀로지,I66472,19066472
7,한국아트라스비엑스,351296,351296
8,한국카앤라이프,IN6515,1902506515
9,한국네트웍스,153529,153529


In [62]:
dfFindBuy = dfTransIdMrgd19[dfTransIdMrgd19['매입회사id'].isna()]
len(dfFindBuy)

1412

In [63]:
dfFindBuy = dfFindBuy.merge(dfLookUp, left_on='매입회사', right_on='firmname', how='left')
dfFindBuy = dfFindBuy[['매입회사', 'firmid']].drop_duplicates()
dfFindBuy

,매입회사,firmid
0,에스프린팅솔루션,1.803405e+09
5,현대다이모스,1.283730e+05
6,현대파워텍,1.794130e+05
19,푸본현대생명보험,NaN
44,엔에스오케이,1.601301e+07
...,...,...
1528,"Aekyung(Shanghai)TradeCo.,Ltd",NaN
1529,PWEFCooperatiefU.A,NaN
1530,COPAEurekaInc,NaN
1531,다우재팬,NaN


In [67]:
print(lookUpListBuy19)

['에스프린팅솔루션', '현대다이모스', '현대파워텍', '푸본현대생명보험', '엔에스오케이', '에스케이해운', '지허브', '에스케이엔카닷컴', '에스케이유화', '에스케이테크엑스', '에스케이증권', 'SKChinaCompany,Ltd', 'SKC&CBeijingCo.,Ltd', 'ESSENCORELimited', 'SKcomputerandcommunicationL.L.C', 'SKMaterials(Jiangsu)Co.,Ltd', 'SKBiotekIrelandLimited', 'SKBiotekUSA,Inc', 'SKUSA,Inc', 'SKE&PCompany', 'SKShanghaiAsphaltCo.,Ltd', 'SKGlobalChemicalInternationalTrading(Shanghai)Co.,Ltd', 'SKGlobalChemical(China)HoldingCo.,Ltd', 'SKGlobalChemicalSingaporePte.Ltd', 'NingboSKPerformanceRubberCo.,Ltd', 'SKGlobalChemicalJapanCo.,Ltd', 'SKPrimacorEurope,S.L.U', 'SKPrimacorAmericasLLC', 'SKSaranAmericasLLC', 'SKGCAmericas,Inc', 'SKGlobalChemicalInternationalTrading(Guangzhou)Co.,Ltd', 'SKEnergyLubricants(Tianjin)Co.,Ltd', 'PT.PatraSK', 'SKLubricantsJapanCo.,Ltd', 'SKLubricantsEuropeB.V', 'SKLubricantsAmericas,Inc', 'SKEnergyInternationalPte,Ltd', 'SKEnergyEurope,Ltd', 'SKEnergyAmericasInc', 'FSKL&S(Shanghai)Co.,Ltd', 'SKhynixSemiconductor(China)Ltd.(SKHYCL)', 'HITECHSemiconducto

In [65]:
dfFindBuy['매입회사'].unique()

array(['에스프린팅솔루션', '현대다이모스', '현대파워텍', '푸본현대생명보험', '엔에스오케이', '에스케이해운',
       '지허브', '에스케이엔카닷컴', '에스케이유화', '에스케이테크엑스', '에스케이증권',
       'ESSENCORELimited', 'SKBiotekIrelandLimited',
       'SKPrimacorAmericasLLC', 'SKSaranAmericasLLC',
       'SKEnergyAmericasInc', 'SKCEuropeGmbH', 'SKGasUSAInc',
       'NanoenTekAmericaInc', 'NanoenTekUSAInc', 'IriverInc', '에이오티',
       'MCNSPolyurethanesMalaysiaSdnBhd', 'MCNSPolyurethanesUSAInc',
       'SKGasTradingLLC', '대전맑은물', '헬로네이처', 'SKNetworksDeutschlandGmbH',
       'SKNetworksHongKongLtd', 'SKNetworksMiddleEastFZE',
       'SKNetworksResourcesAustraliaPtyLtd', 'SKTelesysCorp',
       'SKLubricantsRusLimitedLiabilityCompany', 'SKChemicalsAmerica',
       'SKChemicalsGmbH', 'CelcomPlanetSdnBhd', '실리콘화일', '엘지엔시스', '팜화옹',
       '하우시스이엔지', '루셈', '롯데디에프글로벌', '롯데닷컴', '롯데백화점마산', '롯데시네마', '롯데로지스틱스',
       '롯데아이티테크', '한화에스앤씨', '한화큐셀코리아', '태경화성', 'UniversalBearingsLLC',
       'HanwhaQCELLSAustraliaPtyLtd', 'HanwhaQCELLSGmbh',
       'HANWHATECHMUSA

'거래업체수엘에스는주석참조'

In [70]:
lFindBuy = ['에스프린팅솔루션', '현대다이모스', '현대파워텍', '푸본현대생명보험', '엔에스오케이', '에스케이해운',
             '지허브', '에스케이엔카닷컴', '에스케이유화', '에스케이테크엑스', '에스케이증권', '에이오티',
             '대전맑은물', '헬로네이처', '실리콘화일', '엘지엔시스', '팜화옹', '하우시스이엔지', '루셈',
             '롯데디에프글로벌', '롯데닷컴', '롯데백화점마산', '롯데시네마', '롯데로지스틱스', '롯데아이티테크', 
             '한화에스앤씨', '한화큐셀코리아', '태경화성', '휴먼파워', '에이치앤디코리아', '경주엔바이로', 
             '해양도시가스', '서라벌도시가스', '지에스이엠', '지에스아이티엠', '와이지이', '에이플러스비', 
             '엔씨타스', '현대에너지솔루션', '하이투자증권', '하이자산운용', '몽클레르신세계', '신세계면세점글로벌',
             '한국글로발로지스틱시스템', '씨제이건설', '씨제이헬스케어', '씨제이이앤엠', '씨제이디지털뮤직', '돈돈팜',
             '영우냉동식품', '케이엑스홀딩스', '조이렌트카', '루비콘컴퍼니', '광양항서부컨테이너터미널', '프레시원전북', 
             '미미', '장승포케이블넷', '두산엔진', '디아이피홀딩스', '두타몰', '두산에이엠씨', '델텍', 
             '우성지앤티', '오라통상', 'PCA생명보험', '미래에셋파트너스제구호사모투자', '미래에셋파트너스칠호사모투자',
             '한섬글로벌', '금강에이앤디', '현대엘이디', '이이에치케이이앤에스', '동삼현대한매고분', 
             '데보니안해외자원개발사모투자', '헨리우스코리아', '싱그린FS', '농업회사법인팜앤코', '피오르드프로세싱코리아',
             '유니온', '유니온머티리얼', '플랜에이엔터테인먼트', '밸류포션', '넘버웍스', '럭시', '페이브엔터테인먼트',
             '카닥', '에이디엠이십일', '대원건설산업', '성우종합건설', '온양관광호텔', '삼라희망재단', '삼라자원개발', 
             '한국타이어앤테크놀로지', '한국카앤라이프', '한국네트웍스', '한국테크놀로지그룹', '한국아트라스비엑스', 
             '한국엔지니어링웍스', '한국프리시전웍스', '쇼핑엔티', '고운조경', '이에셋투자개발', '셀트리온화학연', '대우전자', 
             '대우전자서비스', '디비라이텍', '스카이하우징', '스카이건설', '베르디움리빙', '퍼시픽랜드', '호반', '리젠시빌주택',
             '리젠시빌건설', '캠프모바일', '드라마앤컴퍼니', '네무스텍', '아이커넥트', '에피폴라', '우주', '넥스트플로어지하연',
             '넥스트플로어', '엔스토어', '에스비에스에이엔티', '케이씨피', '엔엑스브이피', '엔도어즈', '한라엔컴', '한라엠티스',
             '현대산업', '유진제오호타이탄기업재무안정사모투자', '부산로지스틱스', '대전로지스틱스', '경인로지스틱스', '중부로지스틱스',
             '유진에버베스트턴어라운드기업재무안정사모투자', '크레노바', '에이케이네크워크']

In [81]:
dfFindBuy = pd.DataFrame({'매입회사': lFindBuy})
dfFindBuy.head()

,매입회사
0,에스프린팅솔루션
1,현대다이모스
2,현대파워텍
3,푸본현대생명보험
4,엔에스오케이


In [82]:
dfFindBuy = dfFindBuy.merge(dfLookUp, left_on='매입회사', right_on='firmname', how='left')
dfFindBuy.head()

,매입회사,firmname,firmid
0,에스프린팅솔루션,에스프린팅솔루션,1.803405e+09
1,현대다이모스,현대다이모스,1.283730e+05
2,현대파워텍,현대파워텍,1.794130e+05
3,푸본현대생명보험,NaN,NaN
4,엔에스오케이,엔에스오케이,1.601301e+07


In [83]:
dfFindBuy = dfFindBuy.merge(dfFindSell[['매출회사', '매출회사id']], left_on='매입회사', right_on='매출회사', how='left')
dfFindBuy.head(50)

,매입회사,firmname,firmid,매출회사,매출회사id
0,에스프린팅솔루션,에스프린팅솔루션,1.803405e+09,NaN,NaN
1,현대다이모스,현대다이모스,1.283730e+05,NaN,NaN
2,현대파워텍,현대파워텍,1.794130e+05,NaN,NaN
3,푸본현대생명보험,NaN,NaN,NaN,NaN
4,엔에스오케이,엔에스오케이,1.601301e+07,NaN,NaN
5,에스케이해운,에스케이해운,7.200540e+05,NaN,NaN
6,지허브,지허브,2.101022e+07,NaN,NaN
7,에스케이엔카닷컴,에스케이엔카닷컴,2.306823e+07,NaN,NaN
8,에스케이유화,에스케이유화,3.805930e+05,NaN,NaN
9,에스케이테크엑스,에스케이테크엑스,1.802510e+09,NaN,NaN


In [84]:
dfFindBuy.tail(50)

,매입회사,firmname,firmid,매출회사,매출회사id
92,성우종합건설,성우종합건설,5.174370e+05,NaN,NaN
93,온양관광호텔,온양관광호텔,2.155590e+05,NaN,NaN
94,삼라희망재단,NaN,NaN,NaN,NaN
95,삼라자원개발,삼라자원개발,2.107955e+07,NaN,NaN
96,한국타이어앤테크놀로지,NaN,NaN,한국타이어앤테크놀로지,19066472
97,한국카앤라이프,NaN,NaN,한국카앤라이프,1902506515
98,한국네트웍스,NaN,NaN,한국네트웍스,153529
99,한국테크놀로지그룹,NaN,NaN,한국테크놀로지그룹,351067
100,한국아트라스비엑스,NaN,NaN,한국아트라스비엑스,351296
101,한국엔지니어링웍스,NaN,NaN,한국엔지니어링웍스,767662


In [88]:
dfFindBuy.loc[92, '매출회사']

nan

In [96]:
dfFindBuy['매입회사id'] = dfFindBuy[['firmid', '매출회사id']].apply(lambda x: x['firmid'] if x['firmid'] >= 0 else x['매출회사id'], axis=1)
dfFindBuy = dfFindBuy.drop(columns=['firmname', 'firmid', '매출회사', '매출회사id'])
dfFindBuy

,매입회사,매입회사id
0,에스프린팅솔루션,1803404898.0
1,현대다이모스,128373.0
2,현대파워텍,179413.0
3,푸본현대생명보험,NaN
4,엔에스오케이,16013011.0
...,...,...
137,경인로지스틱스,1803402601.0
138,중부로지스틱스,1803500509.0
139,유진에버베스트턴어라운드기업재무안정사모투자,1801904792
140,크레노바,17023818.0


In [97]:
dfFindBuy = dfFindBuy.set_index('매입회사')
dfFindBuy

,매입회사id
매입회사,
에스프린팅솔루션,1803404898.0
현대다이모스,128373.0
현대파워텍,179413.0
푸본현대생명보험,NaN
엔에스오케이,16013011.0
...,...
경인로지스틱스,1803402601.0
중부로지스틱스,1803500509.0
유진에버베스트턴어라운드기업재무안정사모투자,1801904792


In [99]:
len(dfFindBuy[dfFindBuy['매입회사id'].isna()].index)

19

In [103]:
username = 'wonbok'
password = 'gkrtkrhk5034!'

In [111]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

grsrch_url = 'https://www.kisline.com/cm/CM0100M00GE00.nice'
driver.get(grsrch_url)
driver.find_element(By.CLASS_NAME, 'btn_close_layer').click()
driver.find_element(By.ID, 'lgnuid').send_keys(username)
driver.find_element(By.ID, 'tmp_lgnupassword').click()
driver.find_element(By.ID, 'lgnupassword').send_keys(password)
driver.find_element(By.CLASS_NAME, 'btn_log_in').click()
driver.implicitly_wait(3)
sleep(1)

inputElem = driver.find_element_by_id('q')

for cmpny in dfFindBuy[dfFindBuy['매입회사id'].isna()].index:
    
    inputElem.send_keys(cmpny)
    driver.find_element(By.ID, 'searchView').click()
    driver.implicitly_wait(3)
    try:
        targetRow = driver.find_element(By.XPATH, '//*[@id="eprTable"]/tbody/tr/td[2]')
        kiscode = targetRow.get_attribute('data-kiscode')
        for i in range(len(dfCharToInt)):
            kiscode = re.sub(fr'{dfCharToInt.iloc[i, 0]}', str(dfCharToInt.iloc[i, 1]), kiscode)
        dfFindBuy.loc[cmpny, '매입회사id'] = kiscode
        print(f"Found kiscode for {cmpny}: {kiscode}")
    except:
        print(f"No company found for {cmpny}. Continuing...")
        
    sleep(3)
    inputElem = driver.find_element(By.ID, 'q')
    inputElem.clear()

sleep(3)
driver.find_element_by_link_text('로그아웃').click()
sleep(1)
driver.quit()

Found kiscode for 푸본현대생명보험: 211937
Found kiscode for 에이오티: 19059986
Found kiscode for 롯데아이티테크: 138533
No company found for 한국글로발로지스틱시스템. Continuing...
Found kiscode for 미미: 19055826
No company found for PCA생명보험. Continuing...
No company found for 이이에치케이이앤에스. Continuing...
No company found for 동삼현대한매고분. Continuing...
No company found for 데보니안해외자원개발사모투자. Continuing...
Found kiscode for 삼라희망재단: 21062666
No company found for 디비라이텍. Continuing...
Found kiscode for 호반: 910678
Found kiscode for 드라마앤컴퍼니: 15088665
Found kiscode for 네무스텍: 13020573
Found kiscode for 엔스토어: 2402705560
No company found for 에스비에스에이엔티. Continuing...
Found kiscode for 케이씨피: 221027
Found kiscode for 유진제오호타이탄기업재무안정사모투자: 2101706531
No company found for 에이케이네크워크. Continuing...


In [112]:
dfFindBuy.loc['한국글로발로지스틱시스템', '매입회사id'] = 127586
dfFindBuy.loc['이이에치케이이앤에스', '매입회사id'] = 'JF3529'
dfFindBuy.loc['데보니안해외자원개발사모투자', '매입회사id'] = 'I67185'
dfFindBuy.loc['에스비에스에이엔티', '매입회사id'] = 126820
dfFindBuy.loc['에이케이네크워크', '매입회사id'] = 486454

In [113]:
for i in range(len(dfCharToInt)):
    dfFindBuy['매입회사id'] = dfFindBuy['매입회사id'].replace(dfCharToInt.iloc[i, 0], str(dfCharToInt.iloc[i, 1]), regex=True)
dfFindBuy

,매입회사id
매입회사,
에스프린팅솔루션,1803404898.0
현대다이모스,128373.0
현대파워텍,179413.0
푸본현대생명보험,211937
엔에스오케이,16013011.0
...,...
경인로지스틱스,1803402601.0
중부로지스틱스,1803500509.0
유진에버베스트턴어라운드기업재무안정사모투자,1801904792


In [115]:
dfFindBuy.reset_index()

,매입회사,매입회사id
0,에스프린팅솔루션,1803404898.0
1,현대다이모스,128373.0
2,현대파워텍,179413.0
3,푸본현대생명보험,211937
4,엔에스오케이,16013011.0
...,...,...
137,경인로지스틱스,1803402601.0
138,중부로지스틱스,1803500509.0
139,유진에버베스트턴어라운드기업재무안정사모투자,1801904792
140,크레노바,17023818.0


In [117]:
dfFindSell

,매출회사,firmid,매출회사id
0,현대에너지솔루션,IL9297,1902309297
1,돈돈팜,372671.0,372671.0
2,미래에셋파트너스제구호사모투자,HQ9331,1802809331
3,싱그린FS,467992,467992
4,농업회사법인팜앤코,506576,506576
5,한국테크놀로지그룹,351067,351067
6,한국타이어앤테크놀로지,I66472,19066472
7,한국아트라스비엑스,351296,351296
8,한국카앤라이프,IN6515,1902506515
9,한국네트웍스,153529,153529


In [122]:
dfTransIdMrgdFind19 = dfTransIdMrgd19.merge(dfFindSell[['매출회사', '매출회사id']], on='매출회사', how='left').merge(dfFindBuy, on='매입회사', how='left')
dfTransIdMrgdFind19['매출회사id_x'].nunique()

1341

In [128]:
dfFindSell['매출회사id'].nunique()

14

In [124]:
dfTransIdMrgdFind19['매출회사id'] = dfTransIdMrgdFind19[['매출회사id_x', '매출회사id_y']]\
                                    .apply(lambda x: x['매출회사id_x'] if x['매출회사id_x'] >= 0 else x['매출회사id_y'], axis=1)
dfTransIdMrgdFind19['매출회사id'].nunique()

1355

In [130]:
dfTransIdMrgdFind19['매입회사id_x'].nunique()

1540

In [131]:
dfTransIdMrgdFind19['매입회사id_y'].nunique()

138

In [129]:
dfTransIdMrgdFind19['매입회사id'] = dfTransIdMrgdFind19[['매입회사id_x', '매입회사id_y']]\
                                    .apply(lambda x: x['매입회사id_x'] if x['매입회사id_x'] >= 0 else x['매입회사id_y'], axis=1)
dfTransIdMrgdFind19['매입회사id'].nunique()

1676

In [132]:
dfTransIdMrgdFind19.columns

Index(['매출회사', '매입회사', '매출액', '국내계열사계(매출액)', '해외계열사계(매출액)', '국내매출액', '해외매출액',
       '매출액총계', 'year2', 'year', 'grpname2', '매출회사id_x', '매출사상/비', '매출사금융/비금융',
       '매출액Val', '매입회사id_x', '매입사상/비', '매입사금융/비금융', 'grpname', 'grpcode',
       '매출회사id_y', '매입회사id_y', '매출회사id', '매입회사id'],
      dtype='object')

In [138]:
dfTransIdMrgdFind19.drop(columns=['매출회사id_y', '매입회사id_y', '매출회사id_x', '매입회사id_x'], inplace=True)
dfTransIdMrgdFind19 = dfTransIdMrgdFind19[dfTransIdMrgdFind19['매입회사'] != '거래업체수엘에스는주석참조']

In [148]:
dfTransIdMrgdFind19.to_excel("../data/transactions-mrg/2019/stacked.xlsx", index=False)

# 2020

In [8]:
myApi.getTransTableAll(2020)

===============삼성===============
['삼성전자' '삼성물산' '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스' '삼성디스플레이'
 '삼성코닝어드밴스드글라스' '에스유머티리얼스' '스테코' '세메스' '삼성전자서비스' '삼성전자서비스씨에스' '삼성전자판매'
 '삼성전자로지텍' '수원삼성축' '삼성메디슨' '삼성에스디아이' '삼성전기' '삼성중공업' '에스디플렉스' '제일패션리테일'
 '삼성웰스토리' '시큐아이' '멀티캠퍼스' '에스티엠' '삼성에스디에스' '에스코어' '오픈핸즈' '미라콤아이앤씨'
 '삼성엔지니어링' '에스원' '휴먼티에스에스' '에스원씨알엠' '제일기획' '호텔신라' '신라스테이' '에이치디씨신라면세점'
 '에스비티엠' '삼성경제연' '삼성라이온즈' '삼성화재해상보험' '삼성화재애니카손해사정' '삼성화재서비스손해사정'
 '삼성화재금융서비스보험대리점' '삼성생명보험' '삼성선물' '삼성자산운용' '삼성액티브자산운용' '삼성생명서비스손해사정'
 '삼성에스알에이자산운용' '삼성생명금융서비스보험대리점' '삼성증권' '삼성카드' '삼성카드고객서비스']
['삼성물산' '삼성바이오에피스' '삼성디스플레이' '세메스' '삼성전자서비스' '삼성전자판매' '삼성메디슨' '삼성에스디아이'
 '삼성웰스토리' '삼성에스디에스' '삼성엔지니어링' '에스원' '호텔신라' '삼성화재해상보험' '삼성생명보험' '삼성자산운용'
 '삼성증권' '삼성카드' '삼성전자' '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스'
 '삼성코닝어드밴스드글라스' '에스유머티리얼스' '스테코' '삼성전자서비스씨에스' '삼성전자로지텍' '수원삼성축' '삼성전기'
 '삼성중공업' '제일패션리테일' '시큐아이' '멀티캠퍼스' '에스티엠' '에스코어' '미라콤아이앤씨' '제일기획' '신라스테이'
 '에이치디씨신라면세점' '에스비티엠' '삼성경제연' '삼성라이온즈' '삼성화재애니카손해사정' '삼성화재서비스손해사정' '삼성선물'


['엘지' '엘지화학' '씨텍' '행복누리' '팜한농' '엘지하우시스' '엘지토스템비엠' '그린누리' '엘지생활건강' '코카콜라음료'
 '한국음료' '해태에이치티비' '더페이스샵' '크린소울' '씨앤피코스메틱스' '캐이엔아이' '에프엠지' '엘지파루크' '밝은누리'
 '태극제약' '루치펠로코리아' '엘지엠엠에이' '엘지전자' '엘지디스플레이' '나눔누리' '엘지이노텍' '이노위드' '하이프라자'
 '하이엠솔루텍' '하이텔레서비스' '에이스냉동공조' '하누리' '로보스타' '로보메디' '실리콘웍스' '엘지유플러스' '씨에스리더'
 '아인텔레서비스' '미디어로그' '데이콤크로싱' '씨에스원파트너' '위드유' '엘지씨엔에스' '비즈테크파트너스' '행복마루'
 '에스앤아이코퍼레이션' '곤지암예원' '미래엠' '드림누리' '에스앤아이씨엠' '엘지경영개발원' '엘지스포츠' '지투알'
 '에이치에스애드' '엘베스트' '엘지상사' '판토스' '판토스부산신항물류센터' '헬리스타항공' '한울타리']
['엘지화학' '엘지하우시스' '엘지토스템비엠' '엘지생활건강' '더페이스샵' '엘지파루크' '엘지엠엠에이' '엘지전자'
 '엘지디스플레이' '엘지이노텍' '하이프라자' '엘지히타치워터솔루션' '엘지유플러스' '엘지씨엔에스' '에스앤아이코퍼레이션'
 '엘지경영개발원' '엘지스포츠' '에이치에스애드' '엘지상사' '판토스' 'LGHoldingsJapanCo.,Ltd' '엘지'
 '씨텍' '행복누리' '팜한농' '우지막코리아' '그린누리' '코카콜라음료' '한국음료' '해태에이치티비' '크린소울'
 '씨앤피코스메틱스' '캐이엔아이' '에프엠지' '미젠스토리' '밝은누리' '태극제약' '제이에스제약'
 '울릉추산용천수먹는샘물개발추진' '루치펠로코리아' '울릉샘물' '나눔누리' '이노위드' '하이엠솔루텍' '하이텔레서비스'
 '에이스냉동공조' '하이엔텍' '하누리' '엘지퓨얼셀시스템즈코리아' '로보스타' '로보메디' '실리콘웍스' '씨에스리더'
 '아인텔레서비스' '미디어로그' '데이콤크로

['지에스' '지에스에너지' '지에스파워' '보령엘엔지터미널' '지에스파크이십사' '지에스칼텍스' '지에스엠비즈' '이노폴리텍'
 '상지해운' '지에스바이오' '지에스에코메탈' '지에스리테일' '후레쉬서브' '지에스넷비전' '씨브이에스넷' '지에스네트웍스'
 '에피스' '지에스홈쇼핑' '지에스텔레서비스' '텐바이텐' '지에스글로벌' '피엘에스' '지에스엔텍' '지에스스포츠' '지에스건설'
 '파르나스호텔' '자이에스앤디' '자이에스텍' '지씨에스' '비에스엠' '자이에너지운영' '피앤에쓰' '지앤엠에스테이트'
 '지에스네오텍' '삼양통상' '삼양인터내셔날' '보헌개발' '켐텍인터내셔날' '옥산유통' '경원건설' '승산' '삼정건업'
 '지에스이앤알' '지에스풍력발전']
['지에스에너지' '지에스파워' '지에스파크이십사' '지에스칼텍스' '지에스엠비즈' '지에스바이오' '지에스에코메탈' '지에스리테일'
 '지에스넷비전' '지에스네트웍스' '지에스홈쇼핑' '지에스텔레서비스' '지에스이피에스' '지에스글로벌' '지에스엔텍'
 '지에스스포츠' '지에스건설' '지에스네오텍' '위너셋' '지에스이앤알' '지에스동해전력' '지에스풍력발전' '지에스포천그린에너지'
 '인천종합에너지' '자이에너지운영' '피앤에쓰' '보령엘엔지터미널' '이노폴리텍' '상지해운' '삼양인터내셔날' '켐텍인터내셔날'
 'GSCaltexChinaCo.,Ltd' 'GSCaltexTrading(Suzhou)Co.,Ltd' 'GSCCzechs.r.o'
 'GSC(Suzhou)PlasticsCo.,Ltd' 'GSCaltexIndiaPte.,Ltd'
 'GSCaltex(Langfang)PlasticsCo.,Ltd' 'GSCaltexSingaporePte.,Ltd'
 'GSLubricants(Tianjin)Co.,Ltd' 'GSCaltexMexicoS.deR.L.deC.V'
 'GSGLOBALSINGAPOREPTE.LTD' 'GSGLOBALUSA,INC' 'GSGlobalJapanCo.,LTD'
 'SYENE

['대한항공' '한진칼' '한진' '한국공항' '진에어' '정석기업' '싸이버스카이' '한진정보통신' '부산글로벌물류센터'
 '포항항7부두운영' '칼호텔네트워크' '항공종합서비스' '한진관광' '왕산레저개발' '서울복합물류프로젝트금융투자'
 '서울복합물류자산관리' '아이에이티' '에어코리아' '토파스여행정보' '한국글로발로지스틱스시스템' '한진울산신항운영'
 '한진인천컨테이너터미널' '평택컨테이너터미날' '한진부산컨테이너터미널' '태일통상' '태일캐터링' '청원냉장' '세계혼재항공화물'
 '더블유에이씨항공서비스']
['한진칼' '한진' '한국공항' '진에어' '싸이버스카이' '한진정보통신' '칼호텔네트워크' '항공종합서비스' '제동레저'
 '한진관광' '왕산레저개발' '아이에이티' '에어코리아' '토파스여행정보' '한국글로발로지스틱스시스템' '세계혼재항공화물'
 '더블유에이씨항공서비스' 'HanjinInternationalCorp' 'HanjinInternationalJapan'
 'TotalAviationServiceLLC' 'HanjinIntermodalAmericaInc'
 "WACINT'LLOGISTICSVIETNAMCONPANYLIMITED" '대한항공' '정석기업' '부산글로벌물류센터'
 '서울복합물류자산관리' '한국티비티' '한진울산신항운영' '한진인천컨테이너터미널' '평택컨테이너터미날' '한진부산컨테이너터미널'
 '더원에너지' 'HanjinGlobalLogistics(HongKong)Ltd'
 'HanjinGlobalLogistics(Dalian)Ltd'
 'QINGDAOHANJINLUHAIINTERNATIONALOGISTICSCO.,LTD'
 'HANJINGLOBALLOGISTICS(SHANGHAI)LIMITED'
 'HANJINGLOBALLOGISTICS(SHENZHEN)LIMITED'
 'HANJINGLOBALLOGISTICSEUROPEs.r.o' 'HANJINGLOBALLOGISTICSMYANMARCO.,LTD'
 'HANJINGLOBAL

['미래에셋캐피탈' '미래에셋대우' '미래에셋생명보험' '미래에셋자산운용' '미래에셋벤처투자' '미래에셋펀드서비스'
 '미래에셋금융서비스' '멀티에셋자산운용' '에너지인프라자산운용' '미래에셋파트너스제9호사모투자' '오딘제9차' '미래에셋컨설팅'
 '브랜드무브' '와이케이디벨롭먼트' '육공공']
['미래에셋대우' '미래에셋생명보험' '미래에셋글로벌유니콘사모투자' '미래에셋네이버아시아그로쓰사모투자'
 'MiraeAssetSecurities(Vietnam)LLC' 'MiraeAssetFinanceCompany(Vietnam)'
 '미래에셋캐피탈' '미래에셋자산운용' '미래에셋벤처투자' '멀티에셋자산운용' '미래에셋펀드서비스' '미래에셋금융서비스'
 '에너지인프라자산운용' '미래에셋파트너스9호' '미래에셋대우코리아제일호사모투자' '미래에셋대우글로벌케이그로쓰사모투자'
 '미래에셋대우오퍼튜너티사모투자' '미래에셋대우헌터스사모투자' '미래에셋대우글로벌제일호사모투자' '미래에셋대우코리아제이호사모투자'
 '미래에셋컨설팅' '브랜드무브' '수원학교사랑' '와이케이디벨롭먼트' 'MiraeAssetSecurities(HK)Limited'
 'MiraeAssetSecurities(UK)Ltd' '미래에셋모바일' '서울공항리무진'
 'MiraeAssetGlobalInvestments(HongKong)Ltd'
 'MiraeAssetGlobalInvestments(USA)INC'
 'MiraeAssetGlobalETFsHodlignsLimited' '오딘제9차']
      매출회사                              매입회사      매출액  국내계열사계(매출액)  \
0  미래에셋캐피탈                            미래에셋대우  31736.0      43772.0   
1  미래에셋캐피탈                          미래에셋생명보험   4692.0      43772.0   
2  미래에셋캐피탈                    

...

['코오롱' '코오롱인더스트리' '코오롱플라스틱' '코오롱바스프이노폼' '코오롱머티리얼' '코오롱글로텍' '그린나래'
 '코오롱데크컴퍼지트' '스위트밀' '케이에프엔티' '코오롱글로벌' '네이처브리지' '테크비전' '덕평랜드' '코오롱하우스비전'
 '코오롱환경에너지' '코오롱이엔지니어링' '코오롱생명과학' '코오롱베니트' '엠오디' '코오롱엘에스아이' '코오롱제약'
 '코오롱오토케어서비스' '코오롱오토모티브' '코오롱아우토' '리베토코리아']
['코오롱인더스트리' '코오롱플라스틱' '코오롱머티리얼' '코오롱글로텍' '그린나래' '코오롱데크컴퍼지트' '스위트밀' '씨에이텍'
 '코오롱글로벌' '코오롱하우스비전' '코오롱환경에너지' '코오롱이엔지니어링' '코오롱생명과학' '코오롱베니트' '이노베이스'
 '엠오디' '코오롱엘에스아이' '코오롱제약' '코오롱오토케어서비스' '코오롱오토모티브' '코오롱아우토' '더블유파트너스'
 '코오롱오토플랫폼' '코오롱화이버' '코오롱인베스트먼트' '코오롱' '코오롱바스프이노폼' '케이에프엔티' '네이처브리지'
 '테크비전' '셀빅개발' '데크씨에스피' '양양풍력발전' '리베토코리아' '천안북부일반산업단지']
  매출회사      매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  해외매출액
0  코오롱  코오롱인더스트리  16654.0      43174.0        130.0  60332.0  130.0
1  코오롱   코오롱플라스틱   2751.0      43174.0        130.0  60332.0  130.0
2  코오롱   코오롱머티리얼   2553.0      43174.0        130.0  60332.0  130.0
3  코오롱    코오롱글로텍   3129.0      43174.0        130.0  60332.0  130.0
4  코오롱      그린나래      1.0      43174.0        130.0  60332.0  130.0
   

['DBInc' 'DB메탈' 'DB월드' 'DB하이텍' '동부철' 'DB스탁인베스트' 'DB에프아이에스' 'DB인베스트'
 'DB저축은행' 'DB금융서비스' 'DB금융투자' 'DB생명보험' 'DB손해보험' 'DB씨앤에스자동차손해사정'
 'DB씨에스아이손해사정' 'DB씨에이에스손해사정' 'DB엠앤에스' 'DB자동차보험손해사정' 'DB자산운용' 'DB캐피탈']
['DBInc' 'DB메탈' 'DB월드' 'DB하이텍' 'DB에프아이에스' 'DB저축은행' 'DB금융서비스' 'DB금융투자'
 'DB생명보험' 'DB손해보험' 'DB씨앤에스자동차손해사정' 'DB씨에스아이손해사정' 'DB씨에이에스손해사정' 'DB엠앤에스'
 'DB자동차보험손해사정' 'DB자산운용' 'DB캐피탈' 'DBMetalUSAINC' 'DBMetalEuropeGmbH'
 'PostandTelecomunicationJointStockInsuranceCorporation']
    매출회사      매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액    해외매출액
0  DBInc     DBInc      0.0      56333.0          0.0  165401.0  27583.0
1  DBInc      DB메탈   9024.0      56333.0          0.0  165401.0  27583.0
2  DBInc      DB월드    114.0      56333.0          0.0  165401.0  27583.0
3  DBInc     DB하이텍  11206.0      56333.0          0.0  165401.0  27583.0
4  DBInc  DB에프아이에스   1600.0      56333.0          0.0  165401.0  27583.0
      매출회사                                               매입회사  매출액  \
395  DB캐피탈             

['셀트리온홀딩스' '셀트리온' '셀트리온스킨큐어' '셀트리온제약' '티에스이엔씨']
['셀트리온' '셀트리온스킨큐어' '셀트리온헬스케어' '셀트리온제약' '셀트리온엔터테인먼트' '셀트리온홀딩스']
       매출회사        매입회사       매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  \
0   셀트리온홀딩스        셀트리온     104.0        104.0          NaN     104.0   
1      셀트리온    셀트리온스킨큐어     231.0     871775.0      21800.0  872097.0   
2      셀트리온    셀트리온헬스케어  862574.0     871775.0      21800.0  872097.0   
3      셀트리온      셀트리온제약    8970.0     871775.0      21800.0  872097.0   
4  셀트리온스킨큐어  셀트리온엔터테인먼트     140.0        749.0        163.0   35476.0   

      해외매출액  
0       NaN  
1  109773.0  
2  109773.0  
3  109773.0  
4    4475.0  
        매출회사     매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액   해외매출액
7   셀트리온스킨큐어   셀트리온제약    100.0        749.0        163.0   35476.0  4475.0
8   셀트리온스킨큐어  셀트리온홀딩스      5.0        749.0        163.0   35476.0  4475.0
9     셀트리온제약     셀트리온  21891.0      21891.0          NaN  171469.0  1214.0
10    티에스이엔씨     셀트리온   3033.0       3199.0          NaN    3779.0

['삼천리' '삼천리이엔지' '삼천리이에스' '삼천리모터스' '에스파워' '안산도시개발' '신승에너지' '삼천리열처리'
 '에스티인터내셔널코퍼레이션' '영덕해파랑풍력발전']
['삼천리이엔지' '삼천리이에스' '삼천리모터스' '휴세스' '에스파워' '안산도시개발' '삼천리자산운용' '삼천리' '신승에너지'
 'PT.SAMINDOUTAMAKALTIM' 'PT.TRASINDOMURNIPERKASA' '영덕풍력발전']
  매출회사    매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)      국내매출액  해외매출액
0  삼천리  삼천리이엔지  11685.0      25595.0          NaN  2538242.0    NaN
1  삼천리  삼천리이에스    419.0      25595.0          NaN  2538242.0    NaN
2  삼천리  삼천리모터스      1.0      25595.0          NaN  2538242.0    NaN
3  삼천리     휴세스    910.0      25595.0          NaN  2538242.0    NaN
4  삼천리    에스파워  10224.0      25595.0          NaN  2538242.0    NaN
             매출회사                     매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)  \
25          신승에너지                     에스파워   332.0        332.0          NaN   
26         삼천리열처리                      삼천리    64.0         64.0          NaN   
27  에스티인터내셔널코퍼레이션    PT.SAMINDOUTAMAKALTIM  3612.0          NaN       5717.0   
28  에스티인터내셔널코퍼레이션  PT.TRASINDOMURNIPERKA

['애경유화' '에이케이앤엠엔바이오팜' '에이케이켐텍' '에이케이에셋' '애경화학' '애경산업' '애경에스티' '모두락애경산업'
 '에이제이피' '제주항공' '모두락' '퍼시픽제3호전문사모부동산투자' '제이에이에스' '에이케이에스앤디' '수원애경역사'
 '평택역사' '마포애경타운' '에이엠플러스자산개발' '에이케이아이에스' '애경개발' '에이케이레저' '애경피앤티' '에이텍'
 '코스파' '애드미션' '우영운수' '비컨로지스틱스']
['에이케이앤엠엔바이오팜' '에이케이켐텍' '애경화학' '애경산업' 'Aekyung(Ningbo)ChemicalCo.,Ltd'
 'AEKYUNGHONGKONGCOMPANYLIMITED'
 'AEKYUNG(SHANGHAI)INVESTMENT&TRADINGCO.,LTD' 'AKVINACO.,LTD' '애경유화' '에이텍'
 'AEKYUNG(FOSHAN)PAINTCO.,LTD' 'AKCHEMTECHHANOICO.,Ltd'
 'AEKYUNGCHEMTECHINDIAPRIVATELIMITED' '애경개발' '애경레지콘'
 'AEKYUNGCHEMICAL(QINGDAO)CO.,LTD' '에이케이홀딩스' '한국특수소재' '제주항공' '에이케이에스앤디'
 '수원애경역사' '평택역사' '에이엠플러스자산개발' '에이케이아이에스' '에이케이레저' '애경피앤티' '코스파'
 'Aekyung(Shanghai)TradeCo.,Ltd' '애경에스티' '마포애경타운' '퍼시픽제3호전문사모부동산투자'
 '제이에이에스' '모두락' '에이엠플러스피에프브이기흥' '에이제이피' '애드미션' '에이엠플러스피에프브이산곡' '에이엘오'
 '에이케이에셋']
   매출회사                            매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)  \
0  애경유화                     에이케이앤엠엔바이오팜  3658.0      12917.0      44272.0   
1  애경유화                      

In [143]:
dfTransIdMrgd20, lookUpListSell20, lookUpListBuy20 = myApi.mergeTransId(2020)

=============== 삼성 ===============
[]
['생보부동산신탁', '네추럴나인']
   매출회사      매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액        해외매출액  \
0  삼성전자      삼성물산  110238      2422149  130563681.0  20300939  134471920.0   
1  삼성전자  삼성바이오에피스       4      2422149  130563681.0  20300939  134471920.0   
2  삼성전자   삼성디스플레이  279363      2422149  130563681.0  20300939  134471920.0   
3  삼성전자       세메스       7      2422149  130563681.0  20300939  134471920.0   
4  삼성전자   삼성전자서비스  453084      2422149  130563681.0  20300939  134471920.0   

         매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융       매출액Val  \
0  154772859.0   2020  2019       삼성  380725.0     상장       비금융  154772859.0   
1  154772859.0   2020  2019       삼성  380725.0     상장       비금융  154772859.0   
2  154772859.0   2020  2019       삼성  380725.0     상장       비금융  154772859.0   
3  154772859.0   2020  2019       삼성  380725.0     상장       비금융  154772859.0   
4  154772859.0   2020  2019       삼성  380725.0     상장       비금융  1547728

=============== 엘지 ===============
=============== 롯데 ===============
[]
['동교청기와피에프브이', '바이더웨이', '롯데피에스넷', '롯데첨단소재', '롯데로지스틱스', '현대정보기술', '디시네마오브코리아', '롯데카드', '이비카드', '경기스마트카드', '인천스마트카드', '롯데손해보험', '마이비', '부산하나로카드', '한페이시스', 'VietnamLotteriaCo.,Ltd(vietnam)', 'PT.LOTTERIAINDONESIA', 'JSCLotteRUS', 'PT.SJIndonesia', 'LotteDutyFreeGuam,LLC', 'LottePhuKhanhDutyFreeTradingCo.,Ltd', 'HAITHANH-LOTTECOMPANYLIMITED', 'LotteHotelGuamLLC', 'LotteHotelVietnamCompanyLimited', 'LotteDutyFreeKansaiCo.,Ltd', 'LotteDutyFreeJapanCo.,Ltd', 'LotteDutyFree(Thailand)Co.,Ltd', 'LotteHotelSaintPetersburg,LLC', 'LotteHotelAraiCo.,Ltd', 'LotteHotelHoldingsUSACo.,Ltd', 'LotteHotelNewYorkPalaceLLC', 'LotteHotelVladivostokLLC', 'LotteEuropeInvestmentB.V', '낙천(중국)식품유한공사', 'LOTTEQINGDAOFOODSCO.,LTD.,', '낙천(상해)식품유한공사', 'LotteConfectioneryRUSLLC', 'LotteIndiaCo.,Ltd.(제과)', 'ChocolaterieGuylianN.V', 'LotteKolson(Pvt.)Limited', 'LotteConfectionery(S.E.A.)Pte.Ltd.,', 'RakhatJSC', 'LotteInternationalAmericaCorp', 'Lotte

=============== 지에스 ===============
=============== 현대중공업 ===============
[]
['현대힘스', '현대중공업터보기계', 'HyundaiHeavyIndustriesBrasil-ManufacturingandTradingofConstructionEquipment', 'HyundaiArabiaLLC', '현대중공(중국)투자유한공사', 'HyundaiConstructionEquipmentEuropeN.V', 'HyundaiConstructionEquipmentAmericas,Inc', 'HyundaiConstructionEquipmentIndiaPrivateLtd', '상주현대액압기기유한공사', '현대융자조임유한공사', '현대강소공정기계유한공사', '연대현대빙윤중공유한공사', 'HyundaiHeavyIndustriesCo.Bulgaria', 'HyundaiPowerTransformersUSA,Inc', '현대중공(중국)전기유한공사', 'Hyundai-VietnamShipbuildingCo.,Ltd.', 'HyundaiOilSingaporePte.Ltd', 'HyundaiOilbank(SHANGHAI)CO.,LTD', 'HyundaiGlobalServiceEuropeB.V', 'HyundaiGlobalServiceAmericasCo.,Ltd', 'HyundaiGlobalServiceSingaporePte.Ltd', 'HYUNDAIENERGYSOLUTIONSAMERICAINC', 'HYUNDAIROBOTICS(SHANGHAI)CO.,LTD', 'PT.HyundaiConstructionEquipmentAsia']
     매출회사      매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액      해외매출액  \
0  한국조선해양    현대미포조선  154880       414937       1028.0  421747  2935082.0   
1  한국조선해양   현대삼호중공업  

=============== 엘에스 ===============
=============== 부영 ===============
[]
[]
  매출회사    매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액  해외매출액  매출액총계  year2  \
0   부영    부영주택  3137         4156          NaN   6521    NaN    NaN   2020   
1   부영  동광주택산업    52         4156          NaN   6521    NaN    NaN   2020   
2   부영    동광주택   518         4156          NaN   6521    NaN    NaN   2020   
3   부영    광영토건   255         4156          NaN   6521    NaN    NaN   2020   
4   부영  남광건설산업    30         4156          NaN   6521    NaN    NaN   2020   

   year grpname2    매출회사id 매출사상/비 매출사금융/비금융  매출액Val      매입회사id 매입사상/비  \
0  2019       부영  900311.0    비상장        금융  6521.0  16012860.0    비상장   
1  2019       부영  900311.0    비상장        금융  6521.0    512648.0    비상장   
2  2019       부영  900311.0    비상장        금융  6521.0  16012858.0    비상장   
3  2019       부영  900311.0    비상장        금융  6521.0      7131.0    비상장   
4  2019       부영  900311.0    비상장        금융  6521.0     60687.0    비상장   

  매입사금융/비

=============== 한국투자금융 ===============
=============== 교보생명보험 ===============
[]
['교보생명자산운용(미국)']
     매출회사          매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  해외매출액  \
0  교보생명보험          교보증권   482        16351          NaN  12977036    NaN   
1  교보생명보험      교보악사자산운용  1479        16351          NaN  12977036    NaN   
2  교보생명보험  교보라이프플래닛생명보험    28        16351          NaN  12977036    NaN   
3  교보생명보험     케이씨에이손해사정   575        16351          NaN  12977036    NaN   
4  교보생명보험        교보자산신탁    90        16351          NaN  12977036    NaN   

   매출액총계  year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융      매출액Val  \
0    NaN   2020  2019   교보생명보험  870200.0    비상장        금융  12977036.0   
1    NaN   2020  2019   교보생명보험  870200.0    비상장        금융  12977036.0   
2    NaN   2020  2019   교보생명보험  870200.0    비상장        금융  12977036.0   
3    NaN   2020  2019   교보생명보험  870200.0    비상장        금융  12977036.0   
4    NaN   2020  2019   교보생명보험  870200.0    비상장        금융  12977036.0   

      

=============== 코오롱 ===============
=============== 오씨아이 ===============
[]
['OCI(HongKong)Limited', 'OCIInternationalInc', 'OCI(Shanghai)InternationalTradingLtd', 'UNIDGLOBALPHILS.,Inc', 'OCIUNIDEuropeB.V', 'JIANGSUOCICHEMICALCo.,Ltd', 'eTECARABIALTD', 'eTECMalaysiaSdn.Bhd', '쿼츠테크', '상해개랑가', 'OCIMSdn.Bhd', 'MaSteelOCIChemical', 'OCIVietnam', 'PhilkoPeroxide', 'PhilkoChemLogistics', 'TangshanOCIChemicalCo.,Ltd', 'OCIChinaCo.,Ltd', 'OCISOLARPOWERLLC', 'ShandongOCI', 'ShandongOCI-Jianyangcarbonblack', 'OCISolar(China)Co.,Ltd']
       매출회사                  매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액  \
0      디씨알이                  오씨아이    129.0          129            0     361   
1  유니드글로벌상사                  오씨아이  12398.0        21913        57377  149352   
2  유니드글로벌상사                   유니드   9195.0        21913        57377  149352   
3  유니드글로벌상사                오씨아이페로    320.0        21913        57377  149352   
4  유니드글로벌상사  OCI(HongKong)Limited   7985.0        21913        57377 

=============== 네이버 ===============
=============== 넥슨 ===============
[]
['소호브릭스', '넥슨레드', 'ATMediaInvestor,LLC', 'NEXONCo.,Ltd', 'NEXONAmerica,Inc', 'NEXONMInc', 'NEXON(Thailand)Co.,Ltd', 'NEXONTAIWANLIMITED', 'STOKKEAS', '불리언게임즈', 'BigHugeGames,Inc']
   매출회사    매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액   해외매출액    매출액총계  \
0  엔엑스씨  아퀴스코리아    65         4836       7766.0  13737  8548.0  22285.0   
1  엔엑스씨    가승개발     7         4836       7766.0  13737  8548.0  22285.0   
2  엔엑스씨    가승개발     7         4836       7766.0  13737  8548.0  22285.0   
3  엔엑스씨   넥슨코리아  2774         4836       7766.0  13737  8548.0  22285.0   
4  엔엑스씨     네오플  1568         4836       7766.0  13737  8548.0  22285.0   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융   매출액Val        매입회사id  \
0   2020  2019       넥슨  799173.0    비상장       비금융  22285.0  1.809424e+07   
1   2020  2019       넥슨  799173.0    비상장       비금융  22285.0  1.802408e+09   
2   2020  2019       넥슨  799173.0    비상장       비금융  22285.0  1.8

=============== 태광 ===============
=============== 동원 ===============
[]
['동원냉장', 'StarKistCo', 'SCASA', 'TTP', 'DWGlobalInc', 'DongwonJapanCo.,Ltd', '동원식품(상해)유한공사', 'DongwonF&BVietnamCo.,Ltd', 'StarKistSamoaCo', 'GalapescaS.A', 'CAPSEN', 'KKT', 'TALOFA', 'MVP', '위해삼조유한공사', '상해동부', 'DU', 'DV']
       매출회사    매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액   해외매출액    매출액총계  \
0  동원엔터프라이즈  동원에프앤비  12252        42154       1949.0  43943  1949.0  45892.0   
1  동원엔터프라이즈    동원산업   4892        42154       1949.0  43943  1949.0  45892.0   
2  동원엔터프라이즈  동원시스템즈   4301        42154       1949.0  43943  1949.0  45892.0   
3  동원엔터프라이즈  동원건설산업   1958        42154       1949.0  43943  1949.0  45892.0   
4  동원엔터프라이즈    동원팜스   2018        42154       1949.0  43943  1949.0  45892.0   

   year2  year grpname2    매출회사id 매출사상/비 매출사금융/비금융   매출액Val      매입회사id  \
0   2020  2019       동원  164326.0    비상장       비금융  75885.0    152859.0   
1   2020  2019       동원  164326.0    비상장       비금융  75885.0    100030.0   

[]
['Aekyung(Ningbo)ChemicalCo.,Ltd', 'AEKYUNGHONGKONGCOMPANYLIMITED', 'AEKYUNG(SHANGHAI)INVESTMENT&TRADINGCO.,LTD', 'AKVINACO.,LTD', 'AEKYUNG(FOSHAN)PAINTCO.,LTD', 'AKCHEMTECHHANOICO.,Ltd', 'AEKYUNGCHEMTECHINDIAPRIVATELIMITED', '애경레지콘', 'AEKYUNGCHEMICAL(QINGDAO)CO.,LTD', '한국특수소재', 'Aekyung(Shanghai)TradeCo.,Ltd']
   매출회사                            매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)  \
0  애경유화                     에이케이앤엠엔바이오팜  3658        12917      44272.0   
1  애경유화                          에이케이켐텍   135        12917      44272.0   
2  애경유화                            애경화학  9007        12917      44272.0   
3  애경유화                            애경산업   117        12917      44272.0   
4  애경유화  Aekyung(Ningbo)ChemicalCo.,Ltd  1136        12917      44272.0   

    국내매출액     해외매출액     매출액총계  year2  year grpname2      매출회사id 매출사상/비  \
0  634823  172641.0  807464.0   2020  2019       애경  19066906.0     상장   
1  634823  172641.0  807464.0   2020  2019       애경  19066906.0     상장   
2  634823  

In [144]:
# verify group matching
dfTransIdMrgd20[dfTransIdMrgd20['grpcode'].isna()]

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출회사id,매출사상/비,매출사금융/비금융,매출액Val,매입회사id,매입사상/비,매입사금융/비금융,grpname,grpcode


In [187]:
dfTransIdMrgd20 = dfTransIdMrgd20[dfTransIdMrgd20['매입회사'] != '거래업체수엘에스는주석참조']
dfTransIdMrgd20.loc[:, '매입회사'] = dfTransIdMrgd20['매입회사'].replace('\(\)', '', regex=True)
dfTransIdMrgd20.loc[:, '매입회사'] = dfTransIdMrgd20['매입회사'].replace('\(하림식품\)', '', regex=True)
dfTransIdMrgd20.loc[:, '매입회사'] = dfTransIdMrgd20['매입회사'].replace('\)$', '', regex=True)

In [188]:
dfFindSell = dfTransIdMrgd20[dfTransIdMrgd20['매출회사id'].isna()]
len(dfFindSell)

3

In [189]:
dfFindSell = dfFindSell[['매출회사']].drop_duplicates()
dfFindSell

,매출회사
57,미래에셋파트너스제구호사모투자
188,유진에버베스트턴어라운드기업재무안정사모투자


In [190]:
dfLookUp = pd.read_excel("../data/transactions-mrg/2019/stacked.xlsx")
dfFindSell = dfFindSell.merge(dfLookUp[['매출회사', '매출회사id']], on='매출회사', how='left')
dfFindSell

,매출회사,매출회사id
0,미래에셋파트너스제구호사모투자,1802809331
1,유진에버베스트턴어라운드기업재무안정사모투자,1801904792


In [196]:
dfFindBuy = dfTransIdMrgd20[dfTransIdMrgd20['매입회사id'].isna()][['매입회사']].drop_duplicates()
len(dfFindBuy)

739

In [197]:
dfFindBuy = dfFindBuy[~dfFindBuy['매입회사'].str.contains('[a-zA-Z]+', regex=True)]
dfFindBuy = dfFindBuy.merge(dfLookUp[['매입회사', '매입회사id']].drop_duplicates(), on='매입회사', how='left')
dfFindBuy

,매입회사,매입회사id
0,생보부동산신탁,9.684390e+05
1,네추럴나인,1.906717e+07
2,현대다이모스,1.283730e+05
3,현대파워텍,1.794130e+05
4,아토즈서플라이서비스,1.105050e+05
...,...,...
192,유진에이스제일호사모투자,1.802610e+09
193,유진제삼호오벨리스크기업재무안정사모투자,1.902708e+09
194,유진제오호타이탄기업재무안정사모투자,2.101707e+09
195,유진에버베스트턴어라운드기업재무안정사모투자,1.801905e+09


In [198]:
print(list(dfFindBuy[dfFindBuy['매입회사id'].isna()]['매입회사'].unique()))

['디디아이에스와이230위탁관리부동산투자회사', '디디아이에스씨1338위탁관리부동산투자회사', '디디아이에스씨1339위탁관리부동산투자회사', '디디아이에스씨57위탁관리부동산투자회사', '디디아이레지덴셜제1호위탁관리자부동산투자회사', '낙천(중국)식품유한공사', '낙천(상해)식품유한공사', '낙천(중국)기업관리유한공사', '낙천주업(북경)유한공사', '농업회사법인에치유아이', '낙천공정소료(심양)유한공사', '호석화학무역(상해)유한공사', '낙천공정소료(가흥)유한공사', '낙천공정소료(합비)유한공사', '늑미상무(상해)유한공사', '요녕낙천초시유한공사', '낙천상업관리(천진)유한공사', '낙천백화(심양)유한공사', '낙천백화(천진)유한공사', '낙천백화(성도)유한공사', '낙천신식기술(북경)유한공사', '대홍광고(상해)유한공사', '낙천성건설(북경)유한공사', '낙천지산(성도)유한공사', '서울복합물류프젝트금융투자', '낙천융자조임(중국)유한공사', '한화기계(소유한공사', '푸디스찬음관리(상해)유한공사', '한화토탈화공무역(상해)유한공사', '한화화학(상해)유한공사', '한화국제무역(상해)유한공사', '청도삼양피혁유한공사', '현대중공(중국)투자유한공사', '상주현대액압기기유한공사', '현대융자조임유한공사', '현대강소공정기계유한공사', '연대현대빙윤중공유한공사', '현대중공(중국)전기유한공사', '신세계몰', '제이원', '센트럴관광개발', '에스씨씨오산', '기이가무역(상해)유한공사', '상해이득무역유한공사', '삼해야마코', '농업회사법인제이앤푸드', '스크린엑스', '더원에너지', '케이티씨엔피그로쓰챔프2011의2호사모투자', '동삼현대한매고분', '한섬파리', '현대휘상무역(상해)유한공사', '한우공정기계(상해)유한공사', '제이코믹스', '루프트코리아', '이큐그린제삼차', '드림라인쏠라파워1호', '교보생명자산운용(미국', '코오롱화이버', '상해개랑가', '티에스케이이앤피', '에스엠티케미칼', '코리아엘앤지트레이딩', '기원토건', '동강시스타

In [199]:
lFindBuy = ['디디아이에스와이230위탁관리부동산투자회사', '디디아이에스씨1338위탁관리부동산투자회사', '디디아이에스씨1339위탁관리부동산투자회사',
            '디디아이에스씨57위탁관리부동산투자회사', '디디아이레지덴셜제1호위탁관리자부동산투자회사', '농업회사법인에치유아이', '서울복합물류프젝트금융투자',
            '신세계몰', '제이원', '센트럴관광개발', '에스씨씨오산', '삼해야마코', '농업회사법인제이앤푸드', '스크린엑스', '더원에너지',
            '케이티씨엔피그로쓰챔프2011의2호사모투자', '제이코믹스', '루프트코리아', '이큐그린제삼차', '드림라인쏠라파워1호',
            '코오롱화이버', '티에스케이이앤피', '에스엠티케미칼', '코리아엘앤지트레이딩', '기원토건', '동강시스타', '인프라이니셔티브제1호사모투자', '케이지에네켐']

In [200]:
dfFindBuy

,매입회사,매입회사id
0,생보부동산신탁,9.684390e+05
1,네추럴나인,1.906717e+07
2,현대다이모스,1.283730e+05
3,현대파워텍,1.794130e+05
4,아토즈서플라이서비스,1.105050e+05
...,...,...
192,유진에이스제일호사모투자,1.802610e+09
193,유진제삼호오벨리스크기업재무안정사모투자,1.902708e+09
194,유진제오호타이탄기업재무안정사모투자,2.101707e+09
195,유진에버베스트턴어라운드기업재무안정사모투자,1.801905e+09


In [201]:
dfFindBuy.set_index('매입회사', inplace=True)
dfFindBuy

,매입회사id
매입회사,
생보부동산신탁,9.684390e+05
네추럴나인,1.906717e+07
현대다이모스,1.283730e+05
현대파워텍,1.794130e+05
아토즈서플라이서비스,1.105050e+05
...,...
유진에이스제일호사모투자,1.802610e+09
유진제삼호오벨리스크기업재무안정사모투자,1.902708e+09
유진제오호타이탄기업재무안정사모투자,2.101707e+09


In [202]:
print(lFindBuy)

['디디아이에스와이230위탁관리부동산투자회사', '디디아이에스씨1338위탁관리부동산투자회사', '디디아이에스씨1339위탁관리부동산투자회사', '디디아이에스씨57위탁관리부동산투자회사', '디디아이레지덴셜제1호위탁관리자부동산투자회사', '농업회사법인에치유아이', '서울복합물류프젝트금융투자', '신세계몰', '제이원', '센트럴관광개발', '에스씨씨오산', '삼해야마코', '농업회사법인제이앤푸드', '스크린엑스', '더원에너지', '케이티씨엔피그로쓰챔프2011의2호사모투자', '제이코믹스', '루프트코리아', '이큐그린제삼차', '드림라인쏠라파워1호', '코오롱화이버', '티에스케이이앤피', '에스엠티케미칼', '코리아엘앤지트레이딩', '기원토건', '동강시스타', '인프라이니셔티브제1호사모투자', '케이지에네켐']


In [205]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

grsrch_url = 'https://www.kisline.com/cm/CM0100M00GE00.nice'
driver.get(grsrch_url)
driver.find_element(By.CLASS_NAME, 'btn_close_layer').click()
driver.find_element(By.ID, 'lgnuid').send_keys(username)
driver.find_element(By.ID, 'tmp_lgnupassword').click()
driver.find_element(By.ID, 'lgnupassword').send_keys(password)
driver.find_element(By.CLASS_NAME, 'btn_log_in').click()
driver.implicitly_wait(3)
sleep(1)

inputElem = driver.find_element_by_id('q')

for cmpny in lFindBuy:
    
    if '사모투자' in cmpny:
        srch += cmpny + '전문회사'
    elif '농업회사법인' in cmpny:
        srch = re.sub('농업회사법인', '', cmpny)
    else:
        srch = cmpny
    inputElem.send_keys(srch)
    driver.find_element(By.ID, 'searchView').click()
    driver.implicitly_wait(3)
    try:
        targetRow = driver.find_element(By.XPATH, '//*[@id="eprTable"]/tbody/tr/td[2]')
        kiscode = targetRow.get_attribute('data-kiscode')
        for i in range(len(dfCharToInt)):
            kiscode = re.sub(fr'{dfCharToInt.iloc[i, 0]}', str(dfCharToInt.iloc[i, 1]), kiscode)
        dfFindBuy.loc[cmpny, '매입회사id'] = kiscode
        print(f"Found kiscode for {cmpny}: {kiscode}")
    except:
        print(f"No company found for {cmpny}. Continuing...")
        
    sleep(3)
    inputElem = driver.find_element(By.ID, 'q')
    inputElem.clear()

sleep(3)
driver.find_element_by_link_text('로그아웃').click()
sleep(1)
driver.quit()

Found kiscode for 디디아이에스와이230위탁관리부동산투자회사: 2101809024


WebDriverException: Message: chrome not reachable
  (Session info: chrome=101.0.4951.64)


In [209]:
dfFindBuy.loc['서울복합물류프젝트금융투자', '매입회사id'] = 'H74676'
dfFindBuy.loc['케이티씨엔피그로쓰챔프2011의2호사모투자', '매입회사id'] = 'I66564'
dfFindBuy.loc['코리아엘앤지트레이딩', '매입회사id'] = 'C03079'
dfFindBuy.loc['티에스케이이앤피', '매입회사id'] = 'F14298'
dfFindBuy.loc['인프라이니셔티브제1호사모투자', '매입회사id'] = 'JJ2736'
dfFindBuy.loc['케이지에네켐', '매입회사id'] = 'H81566'

In [210]:
for i in range(len(dfCharToInt)):
    dfFindBuy['매입회사id'] = dfFindBuy['매입회사id'].replace(dfCharToInt.iloc[i, 0], str(dfCharToInt.iloc[i, 1]), regex=True)
dfFindBuy

,매입회사id
매입회사,
생보부동산신탁,968439.0
네추럴나인,19067169.0
현대다이모스,128373.0
현대파워텍,179413.0
아토즈서플라이서비스,110505.0
...,...
유진에이스제일호사모투자,1802609614.0
유진제삼호오벨리스크기업재무안정사모투자,1902708194.0
유진제오호타이탄기업재무안정사모투자,2101706531.0


In [212]:
dfFindBuy.reset_index(inplace=True)

In [213]:
dfFindSell

,매출회사,매출회사id
0,미래에셋파트너스제구호사모투자,1802809331
1,유진에버베스트턴어라운드기업재무안정사모투자,1801904792


In [214]:
dfTransIdMrgdFind20 = dfTransIdMrgd20.merge(dfFindSell, on='매출회사', how='left').merge(dfFindBuy, on='매입회사', how='left')
dfTransIdMrgdFind20.head()

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,...,매출사상/비,매출사금융/비금융,매출액Val,매입회사id_x,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id_y,매입회사id_y
0,삼성전자,삼성물산,110238.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,...,상장,비금융,154772859.0,810053.0,상장,비금융,samsung,511,NaN,NaN
1,삼성전자,삼성바이오에피스,4.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,...,상장,비금융,154772859.0,18082125.0,비상장,비금융,samsung,511,NaN,NaN
2,삼성전자,삼성디스플레이,279363.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,...,상장,비금융,154772859.0,18082126.0,비상장,비금융,samsung,511,NaN,NaN
3,삼성전자,세메스,7.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,...,상장,비금융,154772859.0,314528.0,비상장,비금융,samsung,511,NaN,NaN
4,삼성전자,삼성전자서비스,453084.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,...,상장,비금융,154772859.0,952133.0,비상장,비금융,samsung,511,NaN,NaN


In [215]:
dfTransIdMrgdFind20['매출회사id'] = dfTransIdMrgdFind20[['매출회사id_x', '매출회사id_y']]\
                                    .apply(lambda x: x['매출회사id_x'] if x['매출회사id_x'] >= 0 else x['매출회사id_y'], axis=1)
dfTransIdMrgdFind20['매출회사id'].nunique()

1418

In [216]:
dfTransIdMrgdFind20['매입회사id'] = dfTransIdMrgdFind20[['매입회사id_x', '매입회사id_y']]\
                                    .apply(lambda x: x['매입회사id_x'] if x['매입회사id_x'] >= 0 else x['매입회사id_y'], axis=1)
dfTransIdMrgdFind20['매입회사id'].nunique()

1824

In [217]:
dfTransIdMrgdFind20.drop(columns=['매출회사id_y', '매입회사id_y', '매출회사id_x', '매입회사id_x'], inplace=True)
dfTransIdMrgdFind20.to_excel('../data/transactions-mrg/2020/stacked.xlsx', index=False)

# 2021

In [1]:
import sys
sys.path.append('../scripts')

import api

In [12]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
myApi = api.Api(apiKey)
myApi.getTransTableAll(2021)#, headless=True)

===============삼성===============
No corporation code file found. Downloading the code file...
Corporation Code File downloaded to ../data/raw/opendart/corpCode/CORPCODE.xml
['삼성전자' '삼성물산' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스' '삼성디스플레이' '삼성코닝어드밴스드글라스'
 '에스유머티리얼스' '스테코' '세메스' '삼성전자서비스' '삼성전자서비스씨에스' '삼성전자판매' '삼성전자로지텍' '수원삼성축'
 '삼성메디슨' '삼성에스디아이' '삼성전기' '삼성중공업' '에스디플렉스' '제일패션리테일' '삼성웰스토리' '시큐아이'
 '멀티캠퍼스' '에스티엠' '삼성에스디에스' '에스코어' '오픈핸즈' '미라콤아이앤씨' '삼성엔지니어링' '에스원'
 '휴먼티에스에스' '에스원씨알엠' '제일기획' '호텔신라' '신라스테이' '에스비티엠' '삼성경제연' '삼성라이온즈'
 '삼성화재해상보험' '삼성화재애니카손해사정' '삼성화재서비스손해사정' '삼성화재금융서비스보험대리점' '삼성생명보험' '삼성선물'
 '삼성자산운용' '삼성액티브자산운용' '삼성생명서비스손해사정' '삼성에스알에이자산운용' '삼성생명금융서비스보험대리점' '삼성증권'
 '삼성카드' '삼성카드고객서비스']
['삼성물산' '삼성바이오에피스' '삼성디스플레이' '삼성전자서비스' '삼성전자판매' '삼성메디슨' '삼성에스디아이' '삼성전기'
 '삼성웰스토리' '삼성에스디에스' '삼성엔지니어링' '에스원' '호텔신라' '삼성화재해상보험' '삼성증권' '삼성카드' '삼성전자'
 '서울레이크사이드' '삼우종합건축사사무소' '씨브이네트' '삼성바이오로직스' '삼성코닝어드밴스드글라스' '에스유머티리얼스'
 '스테코' '삼성전자로지텍' '수원삼성축' '삼성중공업' '제일패션리테일' '멀티캠퍼스' '에스티엠' '에스코어' '미라콤아이앤씨'
 '제일기획' '에이치디씨신라면세

KeyboardInterrupt: 

In [3]:
dfTransIdMrgd21, lookUpListSell21, lookUpListBuy21 = myApi.mergeTransId(2021)

=============== 삼성 ===============
[]
[]
   매출회사      매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액        해외매출액  \
0  삼성전자      삼성물산    86705      3142513  141902290.0  19833135  146478056.0   
1  삼성전자  삼성바이오에피스        0      3142513  141902290.0  19833135  146478056.0   
2  삼성전자   삼성디스플레이   196823      3142513  141902290.0  19833135  146478056.0   
3  삼성전자   삼성전자서비스   465778      3142513  141902290.0  19833135  146478056.0   
4  삼성전자    삼성전자판매  2225711      3142513  141902290.0  19833135  146478056.0   

         매출액총계  year2  year grpname2  매출회사id 매출사상/비 매출사금융/비금융        매출액Val  \
0  166311191.0   2021  2020       삼성  380725     상장       비금융  1.660000e+14   
1  166311191.0   2021  2020       삼성  380725     상장       비금융  1.660000e+14   
2  166311191.0   2021  2020       삼성  380725     상장       비금융  1.660000e+14   
3  166311191.0   2021  2020       삼성  380725     상장       비금융  1.660000e+14   
4  166311191.0   2021  2020       삼성  380725     상장       비금융  1.660000e+14   

     매입회사

=============== 롯데 ===============
=============== 한화 ===============
['한화갤러리아']
['한화갤러리아', '한화금융에셋', 'HanwhaMiningServicesAustraliaPty.,Ltd', 'HanwhaChemical(Ningbo)Co.,Ltd', 'HanwhaChemical(Thailand)Co.,Ltd', 'HanwhaQCELLSGmbh', 'HanwhaQCELLSMalaysiaSdn.Bhd', 'PT.HanwhaMiningServicesIndonesia', 'S&PWorldNetworksDMCC', 'HanwhaAZDEL,Inc', 'HanwhaAdvancedMaterialsBeijingCo.,Ltd', 'HanwhaAdvancedMaterialsEurope,s.r.o', 'HanwhaAdvancedMaterialsShanghaiCo.,Ltd', 'HanwhaHongKongCo.,Ltd', 'HanwhaInternationalLLC', 'HanwhaQCELLSJapanCo.,Ltd', 'HanwhaQCELLSChileSpA', 'HanwhaEuropeGmbH', 'HanwhaInternational(S)PteLtd', 'HanwhaAdvancedMaterialsAmericaLLC', 'HanwhaQCELLS(Qidong)Co.,Ltd', 'HanwhaQCELLSUSA,INC', 'HanwhaQCELLSEPCUSA,LLC', 'HanwhaMiningServicesChileSpA', 'HanwhaAdvancedMaterialsGermanyGmbH', 'HanwhaTechwinMiddleEastFZE', 'HanwhaTechwinSecurityVietnam', 'HanwhaQCELLSAmericasHoldings.Corp', '라온태양광', '마루태양광', '미르태양광', '소수태양광', '수리태양광', '아란태양광']
  매출회사        매입회사      매출액  국내계열사계(매출액)  

=============== 씨제이 ===============
=============== 한진 ===============
[]
['제동레저', 'HanjinInternationalCorp', "HanjinInt'lJapanCo.,Ltd", 'TotalAviationServiceLLC', 'HanjinIntermodalAmericaInc', 'QINGDAOHANJINLUHAIINTERNATIONALLOGISTICSCO.,LTD', 'HanjinGlobalLogistics(HongKong)Ltd', 'HANJINGLOBALLOGISTICS(SHANGHAI)LIMITED', 'HanjinGlobalLogistics(Dalian)Ltd', 'HANJINGLOBALLOGISTICS(SHENZHEN)LIMITED', 'HANJINGLOBALLOGISTICSEUROPEs.r.o', 'HANJINGLOBALLOGISTICSMYANMARCO.,LTD', 'HANJINGLOBALLOGISTICS(VIETNAM)LIMITED', 'GlobalLogisticsSystemAsiaPacificCo.,Ltd', "WACINT'LLOGISTICS(HK)LIMITED", "WACINT'LLOGISTICS(QINGDAO)CONPANYLIMITED"]
   매출회사     매입회사      매출액  국내계열사계(매출액)  해외계열사계(매출액)   국내매출액      해외매출액  \
0  대한항공      한진칼   1158.0      65681.0        318.0  736452  6668554.0   
1  대한항공       한진    709.0      65681.0        318.0  736452  6668554.0   
2  대한항공      진에어  56518.0      65681.0        318.0  736452  6668554.0   
3  대한항공     정석기업    141.0      65681.0        318.0  736452  66685

[]
['대림씨엔에스', '대림오토바이']
  매출회사       매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액     해외매출액  \
0   디엘         대림   4843       677705      47040.0  6181343  936791.0   
1   디엘       디엘건설  83395       677705      47040.0  6181343  936791.0   
2   디엘     대림씨엔에스      3       677705      47040.0  6181343  936791.0   
3   디엘  글래드호텔앤리조트   1809       677705      47040.0  6181343  936791.0   
4   디엘       송도파워      2       677705      47040.0  6181343  936791.0   

       매출액총계  year2  year grpname2  매출회사id 매출사상/비 매출사금융/비금융        매출액Val  \
0  7118134.0   2021  2020       DL  500070     상장        금융  7.120000e+12   
1  7118134.0   2021  2020       DL  500070     상장        금융  7.120000e+12   
2  7118134.0   2021  2020       DL  500070     상장        금융  7.120000e+12   
3  7118134.0   2021  2020       DL  500070     상장        금융  7.120000e+12   
4  7118134.0   2021  2020       DL  500070     상장        금융  7.120000e+12   

       매입회사id 매입사상/비 매입사금융/비금융 grpname  grpcode  
0     23705.0    비상장    

[]
['교보생명자산운용(미국)', '교보데이터센터']
     매출회사          매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  해외매출액  \
0  교보생명보험          교보증권   538        16403          NaN  15708927    NaN   
1  교보생명보험      교보악사자산운용  1552        16403          NaN  15708927    NaN   
2  교보생명보험  교보라이프플래닛생명보험    27        16403          NaN  15708927    NaN   
3  교보생명보험        교보자산신탁    91        16403          NaN  15708927    NaN   
4  교보생명보험     케이씨에이손해사정   731        16403          NaN  15708927    NaN   

   매출액총계  year2  year grpname2  매출회사id 매출사상/비 매출사금융/비금융        매출액Val  \
0    NaN   2021  2020   교보생명보험  870200    비상장        금융  1.570000e+13   
1    NaN   2021  2020   교보생명보험  870200    비상장        금융  1.570000e+13   
2    NaN   2021  2020   교보생명보험  870200    비상장        금융  1.570000e+13   
3    NaN   2021  2020   교보생명보험  870200    비상장        금융  1.570000e+13   
4    NaN   2021  2020   교보생명보험  870200    비상장        금융  1.570000e+13   

       매입회사id 매입사상/비 매입사금융/비금융      grpname  grpcode  
0    860115.0     

=============== 넷마블 ===============
=============== 호반건설 ===============
[]
['아브뉴프랑']
   매출회사          매입회사    매출액  국내계열사계(매출액)  해외계열사계(매출액)     국내매출액  해외매출액  \
0  호반건설        호반자산개발  17554       123239          NaN  895802.0    NaN   
1  호반건설        대전용산개발  35115       123239          NaN  895802.0    NaN   
2  호반건설         스카이리빙   6436       123239          NaN  895802.0    NaN   
3  호반건설  스마트시티더원피에프브이   4109       123239          NaN  895802.0    NaN   
4  호반건설   배곧랜드마크피에프브이    207       123239          NaN  895802.0    NaN   

   매출액총계  year2  year grpname2  매출회사id 매출사상/비 매출사금융/비금융        매출액Val  \
0    NaN   2021  2020     호반건설  133127    비상장       비금융  8.960000e+11   
1    NaN   2021  2020     호반건설  133127    비상장       비금융  8.960000e+11   
2    NaN   2021  2020     호반건설  133127    비상장       비금융  8.960000e+11   
3    NaN   2021  2020     호반건설  133127    비상장       비금융  8.960000e+11   
4    NaN   2021  2020     호반건설  133127    비상장       비금융  8.960000e+11   

         매입회사id 매입사상/비 매

[]
['에스비에스에이엔티', '에스비에스international', '케이씨피', '블렌딩', '에스비에스아프리카TV', 'KoreaContentPlatfrom,LCC', '대AMC', 'TSK이앤피', 'TSK코퍼레이션', '울산이앤피', '휴비스워터', 'TSK엔지니어링', 'TSK엔워터테크', 'TSK그린에너지']
     매출회사       매입회사     매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액  해외매출액  매출액총계  \
0  디엠씨미디어      에스비에스   896.0       2978.0          NaN  22732    NaN    NaN   
1  디엠씨미디어    스마트미디어렙  1651.0       2978.0          NaN  22732    NaN    NaN   
2  디엠씨미디어  에스비에스미디어넷     1.0       2978.0          NaN  22732    NaN    NaN   
3  디엠씨미디어   에스비에스엠앤씨    27.0       2978.0          NaN  22732    NaN    NaN   
4  디엠씨미디어   에스비에스엠앤씨    27.0       2978.0          NaN  22732    NaN    NaN   

   year2  year grpname2  매출회사id 매출사상/비 매출사금융/비금융        매출액Val      매입회사id  \
0   2021  2020       태영  234475    비상장       비금융  2.273200e+10    910856.0   
1   2021  2020       태영  234475    비상장       비금융  2.273200e+10  23085463.0   
2   2021  2020       태영  234475    비상장       비금융  2.273200e+10    165071.0   
3   2021  2020       태영  234475 

[]
['IMMInvestmentGroupJapan', 'CrianzaAviationLtd', '구엘', 'PetriniLtd', 'Petrini2Ltd', 'IMMTEPUSLLC', 'TexasBison,LLC', 'CNBVINACO.,LTD', '이미인(상해)화장품유한공사']
         매출회사                  매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)    국내매출액  \
0  아이엠엠인베스트먼트           아이엠엠글로벌사모투자  1795      23626.0         33.0  39341.0   
1  아이엠엠인베스트먼트    아이엠엠스페셜시츄에이션1호사모투자  2421      23626.0         33.0  39341.0   
2  아이엠엠인베스트먼트  아이엠엠스페셜시츄에이션1의2호사모투자    96      23626.0         33.0  39341.0   
3  아이엠엠인베스트먼트      아이엠엠인베스트먼트4호사모투자   115      23626.0         33.0  39341.0   
4  아이엠엠인베스트먼트        아이엠엠인프라제4호사모투자  1255      23626.0         33.0  39341.0   

   해외매출액    매출액총계  year2  year   grpname2  매출회사id 매출사상/비 매출사금융/비금융  \
0  131.0  39472.0   2021  2020  IMM인베스트먼트  162022    비상장        금융   
1  131.0  39472.0   2021  2020  IMM인베스트먼트  162022    비상장        금융   
2  131.0  39472.0   2021  2020  IMM인베스트먼트  162022    비상장        금융   
3  131.0  39472.0   2021  2020  IMM인베스트먼트  162022    비상장        금융   
4  131.0  39

=============== 애경 ===============
=============== 반도홀딩스 ===============
[]
[]
    매출회사  매입회사   매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액  해외매출액  매출액총계  year2  \
0  반도홀딩스  대영개발    98         5742          NaN  30930    NaN    NaN   2021   
1  반도홀딩스  대창개발   450         5742          NaN  30930    NaN    NaN   2021   
2  반도홀딩스  대현개발    37         5742          NaN  30930    NaN    NaN   2021   
3  반도홀딩스  대호개발   130         5742          NaN  30930    NaN    NaN   2021   
4  반도홀딩스  반도건설  3344         5742          NaN  30930    NaN    NaN   2021   

   year grpname2  매출회사id 매출사상/비 매출사금융/비금융        매출액Val    매입회사id 매입사상/비  \
0  2020    반도홀딩스  513164    비상장       비금융  3.093000e+10  21057691    비상장   
1  2020    반도홀딩스  513164    비상장       비금융  3.093000e+10  21057692    비상장   
2  2020    반도홀딩스  513164    비상장       비금융  3.093000e+10  21057683    비상장   
3  2020    반도홀딩스  513164    비상장       비금융  3.093000e+10  21057690    비상장   
4  2020    반도홀딩스  513164    비상장       비금융  3.093000e+10  13015101    비상장

    매출회사    매입회사  매출액  국내계열사계(매출액)  해외계열사계(매출액)  국내매출액  해외매출액  매출액총계  year2  \
0  중앙홀딩스    다보중앙    6      36498.0          NaN  37257    NaN    NaN   2021   
1  중앙홀딩스   미디어링크  174      36498.0          NaN  37257    NaN    NaN   2021   
2  중앙홀딩스  제이콘텐트리  656      36498.0          NaN  37257    NaN    NaN   2021   
3  중앙홀딩스    차이나랩   75      36498.0          NaN  37257    NaN    NaN   2021   
4  중앙홀딩스   필름몬스터   20      36498.0          NaN  37257    NaN    NaN   2021   

   year grpname2    매출회사id 매출사상/비 매출사금융/비금융        매출액Val        매입회사id  \
0  2020       중앙  16006198    비상장       비금융  3.725700e+10  1.600620e+07   
1  2020       중앙  16006198    비상장       비금융  3.725700e+10  1.803100e+09   
2  2020       중앙  16006198    비상장       비금융  3.725700e+10  4.164500e+04   
3  2020       중앙  16006198    비상장       비금융  3.725700e+10  1.803308e+09   
4  2020       중앙  16006198    비상장       비금융  3.725700e+10  1.801306e+09   

  매입사상/비 매입사금융/비금융   grpname  grpcode  
0    비상장       비금융  joongang    36

In [4]:
# verify group matching
dfTransIdMrgd21[dfTransIdMrgd21['grpcode'].isna()]

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출회사id,매출사상/비,매출사금융/비금융,매출액Val,매입회사id,매입사상/비,매입사금융/비금융,grpname,grpcode


In [245]:
dfTransIdMrgd21 = dfTransIdMrgd21[dfTransIdMrgd21['매입회사'] != '거래업체수엘에스는주석참조']
dfTransIdMrgd21 = dfTransIdMrgd21[dfTransIdMrgd21['매입회사'] != '거래해외계열사수효성은주석참조']
dfTransIdMrgd21.loc[:, '매입회사'] = dfTransIdMrgd21['매입회사'].replace('\(\)', '', regex=True)
dfTransIdMrgd21.loc[:, '매입회사'] = dfTransIdMrgd21['매입회사'].replace('\(하림식품\)', '', regex=True)
dfTransIdMrgd21.loc[:, '매입회사'] = dfTransIdMrgd21['매입회사'].replace('\)$', '', regex=True)
dfTransIdMrgd21.loc[:, '매입회사'] = dfTransIdMrgd21['매입회사'].replace('유\)', '', regex=True)
dfTransIdMrgd21.loc[:, '매입회사'] = dfTransIdMrgd21['매입회사'].replace('\(주', '', regex=True)

In [254]:
dfFindSell = dfTransIdMrgd21[dfTransIdMrgd21['매출회사id'].isna()]
len(dfFindSell)

88

In [255]:
dfFindSell = dfFindSell[['매출회사']].drop_duplicates()
dfFindSell

,매출회사
355,에스케이에코플랜트
80,한화갤러리아
57,미래에셋파트너스제구호사모투자
66,광주일보사
14,삼광글라스
42,애경중부컨트리클럽
175,유진에버베스트턴어라운드기업재무안정사모투자
189,인프라이니셔티브제1호사모투자
27,삼양바이오팜
50,메디켐


In [256]:
dfLookUp = pd.read_excel("../data/transactions-mrg/2020/stacked.xlsx")
dfFindSell = dfFindSell.merge(dfLookUp[['매출회사', '매출회사id']], on='매출회사', how='left').drop_duplicates()
dfFindSell

,매출회사,매출회사id
0,에스케이에코플랜트,NaN
1,한화갤러리아,6.200920e+05
46,미래에셋파트너스제구호사모투자,1.802809e+09
47,광주일보사,NaN
48,삼광글라스,3.600660e+05
53,애경중부컨트리클럽,NaN
54,유진에버베스트턴어라운드기업재무안정사모투자,1.801905e+09
56,인프라이니셔티브제1호사모투자,NaN
57,삼양바이오팜,1.807871e+07
59,메디켐,2.842570e+05


In [257]:
dfFindSell.columns

Index(['매출회사', '매출회사id'], dtype='object')

In [258]:
dfFindSell.set_index('매출회사', inplace=True)
dfFindSell.loc['에스케이에코플랜트', '매출회사id'] = 500283
dfFindSell.loc['광주일보사', '매출회사id'] = 'HG4720'
dfFindSell.loc['애경중부컨트리클럽', '매출회사id'] = 'C79298'
dfFindSell.loc['에스케이에코플랜트', '매출회사id'] = 'JJ2736'

In [259]:
for i in range(len(dfCharToInt)):
    dfFindSell['매출회사id'] = dfFindSell['매출회사id'].replace(dfCharToInt.iloc[i, 0], str(dfCharToInt.iloc[i, 1]), regex=True)
dfFindSell.reset_index(inplace=True)
dfFindSell

,매출회사,매출회사id
0,에스케이에코플랜트,2102102736
1,한화갤러리아,620092.0
2,미래에셋파트너스제구호사모투자,1802809331.0
3,광주일보사,1801704720
4,삼광글라스,360066.0
5,애경중부컨트리클럽,13079298
6,유진에버베스트턴어라운드기업재무안정사모투자,1801904792.0
7,인프라이니셔티브제1호사모투자,NaN
8,삼양바이오팜,18078710.0
9,메디켐,284257.0


In [265]:
dfFindBuy = dfTransIdMrgd21[dfTransIdMrgd21['매입회사id'].isna()][['매입회사']].drop_duplicates()
len(dfFindBuy)

616

In [266]:
dfFindBuy = dfFindBuy[~dfFindBuy['매입회사'].str.contains('[a-zA-Z]+', regex=True)]
dfFindBuy = dfFindBuy[~dfFindBuy['매입회사'].str.contains('유한공사', regex=True)]
dfFindBuy = dfFindBuy[~dfFindBuy['매입회사'].str.contains('상해', regex=True)]
dfFindBuy = dfFindBuy[~dfFindBuy['매입회사'].str.contains('재팬', regex=True)]
dfFindBuy = dfFindBuy[~dfFindBuy['매입회사'].str.contains('미국', regex=True)]
dfFindBuy = dfFindBuy[~dfFindBuy['매입회사'].str.contains('파리', regex=True)]
dfFindBuy = dfFindBuy.merge(dfLookUp[['매입회사', '매입회사id']].drop_duplicates(), on='매입회사', how='left')
dfFindBuy

,매입회사,매입회사id
0,현대오트론,381542.0
1,현대엠엔소프트,975583.0
2,에스케이에코플랜트,NaN
3,에스케이바이오랜드,190608.0
4,에스케이와이번스,136740.0
...,...,...
108,엘리움주텍,NaN
109,현대인베스트자산운용,NaN
110,카이트제십육호위탁관리부동산투자회사,NaN
111,카이트제십팔호위탁관리부동산투자회사,NaN


In [267]:
print(list(dfFindBuy[dfFindBuy['매입회사id'].isna()]['매입회사'].unique()))

['에스케이에코플랜트', '디디아이에스에스280위탁관리부동산투자회사', '디디아이와이에스40위탁관리부동산투자회사', '디디아이와이에스832위탁관리부동산투자회사', '영시티위탁관리부동산투자회사', '캐이앤아이', '하나방송', '에피스', '솔루스첨단소재', '프라이싱사모투자', '엑스엔게임즈', '지개남산도시고속화도로', '넥스레드', '대한해운엘엔지', '화진고신장식재료', '울산이앤피', '탈리다쿰', '빈티지랩', '구엘', '애경중부컨트리클럽', '한국초저온', '엘리움주텍', '현대인베스트자산운용', '카이트제십육호위탁관리부동산투자회사', '카이트제십팔호위탁관리부동산투자회사', '트리엔파트너스']


In [13]:
lFindBuy = ['에스케이에코플랜트', '디디아이에스에스280위탁관리부동산투자회사', '디디아이와이에스40위탁관리부동산투자회사', 
            '디디아이와이에스832위탁관리부동산투자회사', '영시티위탁관리부동산투자회사', '캐이앤아이', '하나방송', '에피스', 
            '솔루스첨단소재', '프라이싱사모투자', '엑스엔게임즈', '지개남산도시고속화도로', '넥스레드', '대한해운엘엔지', 
            '화진고신장식재료', '울산이앤피', '탈리다쿰', '빈티지랩', '구엘', '애경중부컨트리클럽', '한국초저온', 
            '엘리움주텍', '현대인베스트자산운용', '카이트제십육호위탁관리부동산투자회사', '카이트제십팔호위탁관리부동산투자회사', '트리엔파트너스']

In [279]:
dfFindBuy.set_index('매입회사',inplace=True)
dfFindBuy

,매입회사id
매입회사,
현대오트론,381542.0
현대엠엔소프트,975583.0
에스케이에코플랜트,NaN
에스케이바이오랜드,190608.0
에스케이와이번스,136740.0
...,...
NaN,762325
NaN,2103106263
NaN,2101902401


In [14]:
username = "wonbok"
password = 'gkrtkrhk5034!'
driver = webdriver.Chrome()
driver.implicitly_wait(3)

grsrch_url = 'https://www.kisline.com/cm/CM0100M00GE00.nice'
driver.get(grsrch_url)
driver.find_element(By.CLASS_NAME, 'btn_close_layer').click()
driver.find_element(By.ID, 'lgnuid').send_keys(username)
driver.find_element(By.ID, 'tmp_lgnupassword').click()
driver.find_element(By.ID, 'lgnupassword').send_keys(password)
driver.find_element(By.CLASS_NAME, 'btn_log_in').click()
driver.implicitly_wait(3)
sleep(1)

inputElem = driver.find_element_by_id('q')

for cmpny in lFindBuy:
    
    if '사모투자' in cmpny:
        srch += cmpny + '전문회사'
    elif '농업회사법인' in cmpny:
        srch = re.sub('농업회사법인', '', cmpny)
    else:
        srch = cmpny
    inputElem.send_keys(srch)
    driver.find_element(By.ID, 'searchView').click()
    driver.implicitly_wait(3)
    try:
        targetRow = driver.find_element(By.XPATH, '//*[@id="eprTable"]/tbody/tr/td[2]')
        kiscode = targetRow.get_attribute('data-kiscode')
        for i in range(len(dfCharToInt)):
            kiscode = re.sub(fr'{dfCharToInt.iloc[i, 0]}', str(dfCharToInt.iloc[i, 1]), kiscode)
        dfFindBuy.loc[cmpny, '매입회사id'] = kiscode
        print(f"Found kiscode for {cmpny}: {kiscode}")
    except:
        print(f"No company found for {cmpny}. Continuing...")
        
    sleep(3)
    inputElem = driver.find_element(By.ID, 'q')
    inputElem.clear()

sleep(3)
driver.find_element_by_link_text('로그아웃').click()
sleep(1)
driver.quit()

Found kiscode for 에스케이에코플랜트: 500283
Found kiscode for 디디아이에스에스280위탁관리부동산투자회사: JH7148
Found kiscode for 디디아이와이에스40위탁관리부동산투자회사: JV2671
Found kiscode for 디디아이와이에스832위탁관리부동산투자회사: JV2741
No company found for 영시티위탁관리부동산투자회사. Continuing...


WebDriverException: Message: chrome not reachable
  (Session info: chrome=101.0.4951.64)


In [281]:
dfFindBuy.loc['캐이앤아이', '매입회사id'] = 762325
dfFindBuy.loc['프라이싱사모투자', '매입회사id'] = 'JS6263'
dfFindBuy.loc['울산이앤피', '매입회사id'] = 'JI2401'
dfFindBuy.loc['엘리움주텍', '매입회사id'] = 'HQ0045'
dfFindBuy.loc['현대인베스트자산운용', '매입회사id'] = 389898

In [282]:
for i in range(len(dfCharToInt)):
    dfFindBuy['매입회사id'] = dfFindBuy['매입회사id'].replace(dfCharToInt.iloc[i, 0], str(dfCharToInt.iloc[i, 1]), regex=True)
dfFindBuy

,매입회사id
매입회사,
현대오트론,381542.0
현대엠엔소프트,975583.0
에스케이에코플랜트,500283
에스케이바이오랜드,190608.0
에스케이와이번스,136740.0
...,...
NaN,762325
NaN,2103106263
NaN,2101902401


In [284]:
dfFindBuy.reset_index(inplace=True)

In [285]:
dfTransIdMrgdFind21 = dfTransIdMrgd21.merge(dfFindSell, on='매출회사', how='left').merge(dfFindBuy, on='매입회사', how='left')
dfTransIdMrgdFind21.head()

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,...,매출사상/비,매출사금융/비금융,매출액Val,매입회사id_x,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id_y,매입회사id_y
0,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,상장,비금융,1.660000e+14,810053.0,상장,비금융,samsung,511,NaN,NaN
1,삼성전자,삼성바이오에피스,0.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,상장,비금융,1.660000e+14,18082125.0,비상장,비금융,samsung,511,NaN,NaN
2,삼성전자,삼성디스플레이,196823.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,상장,비금융,1.660000e+14,18082126.0,비상장,비금융,samsung,511,NaN,NaN
3,삼성전자,삼성전자서비스,465778.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,상장,비금융,1.660000e+14,952133.0,비상장,비금융,samsung,511,NaN,NaN
4,삼성전자,삼성전자판매,2225711.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,상장,비금융,1.660000e+14,102357.0,비상장,비금융,samsung,511,NaN,NaN


In [286]:
dfTransIdMrgdFind21['매출회사id'] = dfTransIdMrgdFind21[['매출회사id_x', '매출회사id_y']]\
                                    .apply(lambda x: x['매출회사id_x'] if x['매출회사id_x'] >= 0 else x['매출회사id_y'], axis=1)
dfTransIdMrgdFind21['매출회사id'].nunique()

1535

In [287]:
dfTransIdMrgdFind21['매입회사id'] = dfTransIdMrgdFind21[['매입회사id_x', '매입회사id_y']]\
                                    .apply(lambda x: x['매입회사id_x'] if x['매입회사id_x'] >= 0 else x['매입회사id_y'], axis=1)
dfTransIdMrgdFind21['매입회사id'].nunique()

1972

In [288]:
dfTransIdMrgdFind21.drop(columns=['매출회사id_y', '매입회사id_y', '매출회사id_x', '매입회사id_x'], inplace=True)
dfTransIdMrgdFind21.to_excel('../data/transactions-mrg/2021/stacked.xlsx', index=False)